### Importing libraries

In [1]:
import math

pixels_width = 1920
pixels_height = 1080
screen_width_inches = 21
screen_height_inches = 11

ppi = math.sqrt(pixels_width**2 + pixels_height**2) / math.sqrt(screen_width_inches**2 + screen_height_inches**2)

print(f"Pixel Per Inch (PPI): {ppi}")



Pixel Per Inch (PPI): 92.92403050255547


In [2]:
import pandas as pd
from collections import defaultdict
import math
import numpy as np
from decimal import Decimal
import scipy.ndimage
import matplotlib.pyplot as plt

import glob


In [3]:
# file_path1 = 'plusLenses/amblypiaStudySanaLeft4.xlsx'
# # file_path2 = 'TestData/amblyopiaStudy MetricsSanaTest.tsv'

# dataExport = pd.read_excel(file_path1)
# dataMetric = pd.read_csv(file_path2, sep='\t')

pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 10)


In [4]:
# dataExport.columns[60:]
# dataExport[['Fixation point X [DACS px]', 'Eye movement type', 'Gaze point X [DACS px]' ]][7800:8100]

In [5]:

# dfSaccade, dataExportWithoutCalibration = preprocessing(dataExport)

In [6]:
# dfSaccade[["Eyetracker timestamp [μs]",  "Presented Stimulus name",  "Eye movement type", "Gaze point X [MCS norm]", "Gaze point Y [MCS norm]", 
#         'Gaze point left X [MCS norm]', 'Gaze point left Y [MCS norm]', 'Gaze point right X [MCS norm]', 
#                          'Gaze point right Y [MCS norm]']].head()

In [7]:
# display(dataExport.columns[0:50])
# print(dataExport['Recording resolution height'])
# print(dataExport['Recording resolution width'])
# print(dataExport['Viewport position X'])
# print(dataExport['Viewport width'])
# print(dataExport['Original Media height'])

Checking to find the end of calibration index

In [8]:
from sklearn.impute import KNNImputer

def preprocessing(dataExport):
    imputer = KNNImputer(n_neighbors=2)

    #dataExport[dataExport.eq("Eyetracker Calibration").any(1)]['Presented Stimulus name']
#     calibrationEnd = dataExport[dataExport.eq("Eyetracker Calibration").any(1)]['Presented Stimulus name'].index.to_list()[-1]
#     calibrationEnd+=1
    calibration_rows = dataExport[dataExport['Presented Stimulus name'].eq("Eyetracker Calibration")]

    # Get the index of the last row where 'Event' column equals "Eyetracker Calibration"
    calibrationEnd = calibration_rows.index[-1] + 1
    
    dataExport = dataExport[["Eyetracker timestamp [μs]", "Event", "Event value", "Gaze point X [MCS norm]", "Gaze point Y [MCS norm]", "Eye movement type", 
    "Presented Stimulus name", 'AOI hit [slothStimulus-final (1) - sloth1]',
       'AOI hit [slothStimulus-final (1) - sloth2]',
       'AOI hit [slothStimulus-final (1) - sloth3]',
       'AOI hit [slothStimulus-final (1) - sloth4]',
       'AOI hit [slothStimulus-final (1) - sloth5]',
       'AOI hit [slothStimulus-final (1) - sloth6]',
       'AOI hit [slothStimulus-final (1) - sloth7]',
       'AOI hit [slothStimulus-final (1) - sloth8]',
       'AOI hit [slothStimulus-final (1) - sloth9]',
       'AOI hit [slothStimulus-final (1) - sloth10]',
       'AOI hit [slothStimulus-final (1) - sloth11]',
       'AOI hit [catStimulus-final (1) - cat1]',
        'AOI hit [catStimulus-final (1) - cat2]',
       'AOI hit [catStimulus-final (1) - cat3]',
       'AOI hit [catStimulus-final (1) - cat4]',
       'AOI hit [catStimulus-final (1) - cat5]',
       'AOI hit [catStimulus-final (1) - cat6]',
       'AOI hit [catStimulus-final (1) - cat7]',
       'AOI hit [catStimulus-final (1) - cat8]',
       'AOI hit [catStimulus-final (1) - cat9]',
       'AOI hit [catStimulus-final (1) - cat10]',
       'AOI hit [catStimulus-final (1) - cat11]',
        'Gaze point left X [MCS norm]', 'Gaze point left Y [MCS norm]', 'Gaze point right X [MCS norm]', 
                         'Gaze point right Y [MCS norm]', 
        'Gaze point left X [DACS px]', 'Gaze point left Y [DACS px]', 'Gaze point right X [DACS px]', 
                         'Gaze point right Y [DACS px]',  'Gaze point X [DACS px]',  'Gaze point Y [DACS px]'
                            ]]



    #Only isolate non calibration data
    dataExportWithoutCalibration = dataExport[calibrationEnd:]

    #Drop all columns with eye movement type of eyes not found
    dataExportWithoutCalibration = dataExportWithoutCalibration[dataExportWithoutCalibration['Eye movement type'] != 'EyesNotFound']

    #Convert the x and y coordinates into pixels and then into inches using ppi of 92.
    
    #Check this -
    #Normalized --> absolute pixels on the screen - ()
    dataExportWithoutCalibration[[ 'Gaze point X [MCS norm]', 'Gaze point left X [MCS norm]', 'Gaze point right X [MCS norm]']] = dataExportWithoutCalibration[[ 'Gaze point X [MCS norm]', 'Gaze point left X [MCS norm]', 'Gaze point right X [MCS norm]']]*1920/92 

    dataExportWithoutCalibration[[ 'Gaze point Y [MCS norm]', 'Gaze point left Y [MCS norm]', 'Gaze point right Y [MCS norm]']] = dataExportWithoutCalibration[[ 'Gaze point Y [MCS norm]', 'Gaze point left Y [MCS norm]', 'Gaze point right Y [MCS norm]']]*1080/92

    #Find the first timestamp that isn't nan: 
    # nonNanTimeStampSlothStart = dataExportWithoutCalibration['Eyetracker timestamp [μs]'].iloc[0]
    nonNanTimeStampSlothStart = dataExportWithoutCalibration.loc[dataExportWithoutCalibration['Eyetracker timestamp [μs]'].first_valid_index(), 'Eyetracker timestamp [μs]']

    #normalizes TIME when the calibration ends and stimulus sloth starts (at 0)
    dataExportWithoutCalibration['Eyetracker timestamp [μs]'] = dataExportWithoutCalibration['Eyetracker timestamp [μs]'] - nonNanTimeStampSlothStart #dataExportWithoutCalibration['Eyetracker timestamp [μs]'].iloc[0]

    #normalizes TIME when the sloth ends and cats start (at 0)
    #find the index where the cat stimulus begins
    indices = dataExportWithoutCalibration.index[dataExportWithoutCalibration['Presented Stimulus name'] == "catStimulus-final (1)"]

    #Find the beginning time for normalization
    beginTime = dataExportWithoutCalibration['Eyetracker timestamp [μs]'].loc[indices[0]]
    index_of_catBegin = (dataExportWithoutCalibration.index.tolist()).index(indices[0])
    onylCat = dataExportWithoutCalibration.index[index_of_catBegin:]
    #iterate through all times for cat stimulus and update
    for i in onylCat:
        dataExportWithoutCalibration.loc[i, 'Eyetracker timestamp [μs]'] = dataExportWithoutCalibration.loc[i, 'Eyetracker timestamp [μs]'] - beginTime

    

    #interpolate data
    columns_to_interpolate = ['Gaze point X [MCS norm]', 'Gaze point Y [MCS norm]', 'Gaze point left X [MCS norm]', 
                          'Gaze point left Y [MCS norm]', 'Gaze point right X [MCS norm]', 'Gaze point right Y [MCS norm]', 
                              'Gaze point left X [DACS px]', 'Gaze point left Y [DACS px]', 'Gaze point right X [DACS px]', 
                         'Gaze point right Y [DACS px]',  'Gaze point X [DACS px]',  'Gaze point Y [DACS px]'
                             ]
    
    

    # Use the subset parameter to interpolate only the specified columns

    dataExportWithoutCalibration[columns_to_interpolate] = dataExportWithoutCalibration[columns_to_interpolate].interpolate(method='polynomial', order =2 )
#     dataExportWithoutCalibration['Gaze point right X [MCS norm]'] = imputer.fit_transform(dataExportWithoutCalibration[['Gaze point right X [MCS norm]']])
#     dataExportWithoutCalibration['Gaze point right Y [MCS norm]'] = imputer.fit_transform(dataExportWithoutCalibration[['Gaze point right X [MCS norm]']])


    nan_df = dataExportWithoutCalibration.isna().sum()  # or df.isnull()
    print("LOOK HERE")
    print(nan_df)
    
#     display(dataExportWithoutCalibration['Gaze point right X [MCS norm]'][0:100])
    #isolate only saccade data
    mask = dataExportWithoutCalibration["Eye movement type"] == "Saccade"
    dfSaccade = dataExportWithoutCalibration[mask]
    excel_file_name = "thing"
#     dfSaccade.to_excel(os.path.join(os.getcwd(), excel_file_name), index=False)
#     dfSaccade.to_excel(os.path.join(os.getcwd(), excel_file_name + ".xlsx"), index=False, engine='xlsxwriter')

    
#     display(dfSaccade.head())
#     display(dfSaccade.tail())
    return dfSaccade, dataExportWithoutCalibration



In [9]:
def primaryAndSecondarySaccade(dfSaccade, times_dict): 
    AOI_list = ['AOI hit [slothStimulus-final (1) - sloth1]',
           'AOI hit [slothStimulus-final (1) - sloth2]',
           'AOI hit [slothStimulus-final (1) - sloth3]',
           'AOI hit [slothStimulus-final (1) - sloth4]',
           'AOI hit [slothStimulus-final (1) - sloth5]',
           'AOI hit [slothStimulus-final (1) - sloth6]',
           'AOI hit [slothStimulus-final (1) - sloth7]',
           'AOI hit [slothStimulus-final (1) - sloth8]',
           'AOI hit [slothStimulus-final (1) - sloth9]',
           'AOI hit [slothStimulus-final (1) - sloth10]',
           'AOI hit [slothStimulus-final (1) - sloth11]',
           'AOI hit [catStimulus-final (1) - cat1]',
            'AOI hit [catStimulus-final (1) - cat2]',
           'AOI hit [catStimulus-final (1) - cat3]',
           'AOI hit [catStimulus-final (1) - cat4]',
           'AOI hit [catStimulus-final (1) - cat5]',
           'AOI hit [catStimulus-final (1) - cat6]',
           'AOI hit [catStimulus-final (1) - cat7]',
           'AOI hit [catStimulus-final (1) - cat8]',
           'AOI hit [catStimulus-final (1) - cat9]',
           'AOI hit [catStimulus-final (1) - cat10]',
          'AOI hit [catStimulus-final (1) - cat11]']

    aoi_cat =  ['AOI hit [catStimulus-final (1) - cat1]',
            'AOI hit [catStimulus-final (1) - cat2]',
           'AOI hit [catStimulus-final (1) - cat3]',
           'AOI hit [catStimulus-final (1) - cat4]',
           'AOI hit [catStimulus-final (1) - cat5]',
           'AOI hit [catStimulus-final (1) - cat6]',
           'AOI hit [catStimulus-final (1) - cat7]',
           'AOI hit [catStimulus-final (1) - cat8]',
           'AOI hit [catStimulus-final (1) - cat9]',
           'AOI hit [catStimulus-final (1) - cat10]',
          'AOI hit [catStimulus-final (1) - cat11]']




    primarySaccadeIndex = defaultdict(int)
    primarySaccadeTime = defaultdict(int)
    dfSaccadeReset = dfSaccade.reset_index()
    index_of_catBegin = dfSaccadeReset[dfSaccadeReset['Presented Stimulus name'] == "catStimulus-final (1)"].iloc[0, 0]
    

    for i in times_dict.keys():
        for index, row in dfSaccade.iterrows():
            time = row['Eyetracker timestamp [μs]']
            if i in aoi_cat:
                if index > index_of_catBegin: 
                    if time/1000000 > (times_dict[i]) and (time/1000000 < (times_dict[i] + 2.5)): 
                        primarySaccadeIndex[i] = index
                        primarySaccadeTime[i] = time/1000000
                        break


            else: 
                if time/1000000 > (times_dict[i]) and (time/1000000 < (times_dict[i] + 2.5)): 
                    primarySaccadeIndex[i] = index
                    primarySaccadeTime[i] = time/1000000
                    break
    primarySaccadeTime
   


    ## TIME + AOI SECONDARY SACCADES
    secondarySaccadesTimeAndAOI = defaultdict(int)
    for i in times_dict.keys():
        all_possible_hits = dfSaccade[dfSaccade[i] == np.max(dfSaccade[i])]
        for index, row in all_possible_hits.iterrows(): 
            if i in aoi_cat:
                if index > index_of_catBegin:
                    time = row['Eyetracker timestamp [μs]']
                    if (time/1000000 > (primarySaccadeTime[i]+0.2)) and (time/1000000 < (times_dict[i] + 2.5)):
                        secondarySaccadesTimeAndAOI[i] = index
                        break
            else:
                time = row['Eyetracker timestamp [μs]']
                if (time/1000000 > (primarySaccadeTime[i]+0.2)) and (time/1000000 < (times_dict[i] + 2.5)):
                    secondarySaccadesTimeAndAOI[i] = index
                    break
#     display(secondarySaccadesTimeAndAOI)
#     display(primarySaccadeIndex)


    secondarySaccadesTime = defaultdict(int)
    for i in times_dict.keys():
        for index, row in dfSaccade.iterrows():
            if i in aoi_cat:
                if index > index_of_catBegin:
                    time = row['Eyetracker timestamp [μs]']
                    if (time/1000000 > (0.200 + primarySaccadeTime[i])) and (time/1000000 < (times_dict[i] + 2.5)):
                        secondarySaccadesTime[i] = index
                        break
            else:
                time = row['Eyetracker timestamp [μs]']
                if (time/1000000 > (0.200 + primarySaccadeTime[i])) and (time/1000000 < (times_dict[i] + 2.5)):
                    secondarySaccadesTime[i] = index
                    break

    return primarySaccadeTime, primarySaccadeIndex, secondarySaccadesTime, secondarySaccadesTimeAndAOI


In [10]:
# dataExport[calibrationEnd:]
# dataExport['Eyetracker timestamp [μs]'].isna().sum()
# dataExport['Eyetracker timestamp [μs]'].count()
# dfSaccade['Eyetracker timestamp [μs]'].isna().sum()

### Identify the primary saccade 

In [11]:
# ppi = math.sqrt(1080**2 + 1920**2) / 24
# print(ppi)
# viewing_dist = 22*ppi
# print(viewing_dist)

#convert the normalizaed values into pixels and then into 
#we have l is 22 inches


In [12]:
def find_closest_values(lst, target):
    closest_lower = None
    closest_higher = None

    for value in lst:
        if value <= target:
            if closest_lower is None or target - value < target - closest_lower:
                closest_lower = value
        if value >= target:
            if closest_higher is None or value - target < closest_higher - target:
                closest_higher = value

    return closest_lower, closest_higher


#### Finding amplitude for both eyes, right, eye, left eye, for primary saccades. 

In [13]:
def amplitudePrimarySaccade(dfSaccade, primarySaccadeIndex, dataExportWithoutCalibration): 
    df_reset = dfSaccade.reset_index()
    df_reset['index'] = df_reset['index'].astype(int)

    df_reset_orginal = dataExportWithoutCalibration.reset_index()
    df_reset_orginal['index'] = df_reset_orginal['index'].astype(int)

    amplitudeBothEyesDict = defaultdict(int)
    amplitudeLeftEyeDict = defaultdict(int)
    amplitudeRightEyeDict = defaultdict(int)


    for key,value in primarySaccadeIndex.items():
        fixation_list = dataExportWithoutCalibration
        mask = dataExportWithoutCalibration["Eye movement type"] == "Fixation"
        dfFixation = dataExportWithoutCalibration[mask]
        
        fix1, fix2 = find_closest_values(dfFixation.index.tolist(), value) 
        if fix1 is None: 
            fix1 = value


        if np.isnan(value)==False:
            row1 = df_reset_orginal[df_reset_orginal['index'] == fix1]
#             display(df_reset_orginal)
            x1 = Decimal(str(row1['Gaze point X [DACS px]'].iloc[0]))
            y1 = Decimal(str(row1['Gaze point Y [DACS px]'].iloc[0]))

            #convert the index label to actual index
            
#             index_of_desired_row = df_reset_orginal[df_reset_orginal['index'] == fixation1].index
#             print("start")
#             print(index_of_desired_row)
#             index_of_desired_row = index_of_desired_row -1
#             new_value = df_reset_orginal.loc[index_of_desired_row, 'index']
#             print(new_value)
            row2 = df_reset_orginal[df_reset_orginal['index'] == int(fix2)]
            x2 = Decimal(str(row2.loc[:,'Gaze point X [DACS px]'].iloc[0]))
            y2 = Decimal(str(row2.loc[:,'Gaze point Y [DACS px]'].iloc[0]))

            deltaX = x1-x2
            deltaY = y1-y2

            distance = math.sqrt((deltaX**2) + (deltaY**2))
            amplitudeBothEyesDict[key] = (180/np.pi) * np.arctan(distance/(28*92))

            x1 = row1['Gaze point left X [DACS px]'].iloc[0]
            y1 = row1['Gaze point left Y [DACS px]'].iloc[0]
            x2 = row2.loc[:,'Gaze point left X [DACS px]'].iloc[0]
            y2 = row2.loc[:,'Gaze point left Y [DACS px]'].iloc[0]
            deltaX = x1-x2
            deltaY = y1-y2
            distance = math.sqrt((deltaX**2) + (deltaY**2))
            
            amplitudeLeftEyeDict[key] = (180/np.pi) * np.arctan(distance/(28*92))
            
            
            
            x1 = row1['Gaze point right X [DACS px]'].iloc[0]
            y1 = row1['Gaze point right Y [DACS px]'].iloc[0]
            x2 = row2.loc[:,'Gaze point right X [DACS px]'].iloc[0]
            y2 = row2.loc[:,'Gaze point right Y [DACS px]'].iloc[0]
            print(x1,y1,x2,y2)
            deltaX = x1-x2
            deltaY = y1-y2
            distance = math.sqrt((deltaX**2) + (deltaY**2))
            amplitudeRightEyeDict[key] = (180/np.pi) * np.arctan(distance/(28*92))


   
    
    print("Primary Saccade Amplitude for Both Eyes (Mean and STD)")
    values = list(amplitudeBothEyesDict.values())
    print(np.mean(values), np.std(values))
    print(" ")
    print("Primary Saccade Amplitude for Left Eyes (Mean and STD)")
    values = list(amplitudeLeftEyeDict.values())
    print(np.mean(values), np.std(values))
    print(" ")
    print("Primary Saccade Amplitude for Right Eyes (Mean and STD)")
    values = list(amplitudeRightEyeDict.values())
    print(np.mean(values), np.std(values))
    print(" ")
    
    display(amplitudeBothEyesDict)
    print("RIGHT")
    display(amplitudeRightEyeDict)
    print(" ")
    print("LEFT")
    display(amplitudeLeftEyeDict)

    return amplitudeBothEyesDict, amplitudeRightEyeDict, amplitudeLeftEyeDict

### Finding amplitude for both eyes, right eye, left eye for secondary saccades 



### Secondary saccades (only time constraint)

In [14]:
def secondarySaccadeTimeAmplitude(dfSaccade, dataExportWithoutCalibration, secondarySaccadesTime): 
    df_reset = dfSaccade.reset_index()
    df_reset['index'] = df_reset['index'].astype(int)

    df_reset_orginal = dataExportWithoutCalibration.reset_index()
    df_reset_orginal['index'] = df_reset_orginal['index'].astype(int)

    amplitudeBothEyesDict = defaultdict(int)
    amplitudeLeftEyeDict = defaultdict(int)
    amplitudeRightEyeDict = defaultdict(int)


    for key,value in secondarySaccadesTime.items():
        fixation_list = dataExportWithoutCalibration
        mask = dataExportWithoutCalibration["Eye movement type"] == "Fixation"
        dfFixation = dataExportWithoutCalibration[mask]
        fix1, fix2 = find_closest_values(dfFixation.index.tolist(), value) 
        if fix1 is None: 
            fix1 = value
        if np.isnan(value)==False:
            
            row1 = df_reset_orginal[df_reset_orginal['index'] == fix1]
            x1 = Decimal(str(row1['Gaze point X [DACS px]'].iloc[0]))
            y1 = Decimal(str(row1['Gaze point Y [DACS px]'].iloc[0]))

            #convert the index label to actual index
            
#             index_of_desired_row = df_reset_orginal[df_reset_orginal['index'] == fixation1].index
#             print("start")
#             print(index_of_desired_row)
#             index_of_desired_row = index_of_desired_row -1
#             new_value = df_reset_orginal.loc[index_of_desired_row, 'index']
#             print(new_value)
            row2 = df_reset_orginal[df_reset_orginal['index'] == int(fix2)]
            x2 = Decimal(str(row2.loc[:,'Gaze point X [DACS px]'].iloc[0]))
            y2 = Decimal(str(row2.loc[:,'Gaze point Y [DACS px]'].iloc[0]))

            deltaX = x1-x2
            deltaY = y1-y2

            distance = math.sqrt((deltaX**2) + (deltaY**2))
            amplitudeBothEyesDict[key] = (180/np.pi) * np.arctan(distance/(28*92))

            x1 = row1['Gaze point left X [DACS px]'].iloc[0]
            y1 = row1['Gaze point left Y [DACS px]'].iloc[0]
            x2 = row2.loc[:,'Gaze point left X [DACS px]'].iloc[0]
            y2 = row2.loc[:,'Gaze point left Y [DACS px]'].iloc[0]
            deltaX = x1-x2
            deltaY = y1-y2
            distance = math.sqrt((deltaX**2) + (deltaY**2))
            
            amplitudeLeftEyeDict[key] = (180/np.pi) * np.arctan(distance/(28*92))
            
            

            x1 = row1['Gaze point right X [DACS px]'].iloc[0]
            y1 = row1['Gaze point right Y [DACS px]'].iloc[0]
            x2 = row2.loc[:,'Gaze point right X [DACS px]'].iloc[0]
            y2 = row2.loc[:,'Gaze point right Y [DACS px]'].iloc[0]
            deltaX = x1-x2
            deltaY = y1-y2
            distance = math.sqrt((deltaX**2) + (deltaY**2))
            amplitudeRightEyeDict[key] = (180/np.pi) * np.arctan(distance/(28*92))




    print("Secondary Saccade Amplitude for Both Eyes (Mean and STD)")
    values = list(amplitudeBothEyesDict.values())
    print(np.mean(values), np.std(values))
    print(" ")
    print("Secondary Saccade Amplitude for Left Eyes (Mean and STD)")
    values = list(amplitudeLeftEyeDict.values())
    print(np.mean(values), np.std(values))
    print(" ")
    print("Secondary Saccade Amplitude for Right Eyes (Mean and STD)")
    values = list(amplitudeRightEyeDict.values())
    print(np.mean(values), np.std(values))
    print(" ")
   
    return amplitudeBothEyesDict, amplitudeRightEyeDict, amplitudeLeftEyeDict

### Secondary saccades (time and AOI constraint)

In [15]:
def secondarySaccadeTimeandAOIAmplitude(dfSaccade, dataExportWithoutCalibration, secondarySaccadesTimeAndAOI):
    df_reset = dfSaccade.reset_index()
    df_reset['index'] = df_reset['index'].astype(int)

    df_reset_orginal = dataExportWithoutCalibration.reset_index()
    df_reset_orginal['index'] = df_reset_orginal['index'].astype(int)

    amplitudeBothEyesDict = defaultdict(int)
    amplitudeLeftEyeDict = defaultdict(int)
    amplitudeRightEyeDict = defaultdict(int)


    for key,value in secondarySaccadesTimeAndAOI.items():
        fixation_list = dataExportWithoutCalibration
        mask = dataExportWithoutCalibration["Eye movement type"] == "Fixation"
        dfFixation = dataExportWithoutCalibration[mask]
        fix1, fix2 = find_closest_values(dfFixation.index.tolist(), value) 
        if fix1 is None: 
            fix1 = value

        if np.isnan(value)==False:
            
            row1 = df_reset_orginal[df_reset_orginal['index'] == fix1]
            x1 = Decimal(str(row1['Gaze point X [DACS px]'].iloc[0]))
            y1 = Decimal(str(row1['Gaze point Y [DACS px]'].iloc[0]))

            #convert the index label to actual index
            
#             index_of_desired_row = df_reset_orginal[df_reset_orginal['index'] == fixation1].index
#             print("start")
#             print(index_of_desired_row)
#             index_of_desired_row = index_of_desired_row -1
#             new_value = df_reset_orginal.loc[index_of_desired_row, 'index']
#             print(new_value)
            row2 = df_reset_orginal[df_reset_orginal['index'] == int(fix2)]
            x2 = Decimal(str(row2.loc[:,'Gaze point X [DACS px]'].iloc[0]))
            y2 = Decimal(str(row2.loc[:,'Gaze point Y [DACS px]'].iloc[0]))

            deltaX = x1-x2
            deltaY = y1-y2

            distance = math.sqrt((deltaX**2) + (deltaY**2))
            amplitudeBothEyesDict[key] = (180/np.pi) * np.arctan(distance/(28*92))

            x1 = row1['Gaze point left X [DACS px]'].iloc[0]
            y1 = row1['Gaze point left Y [DACS px]'].iloc[0]
            x2 = row2.loc[:,'Gaze point left X [DACS px]'].iloc[0]
            y2 = row2.loc[:,'Gaze point left Y [DACS px]'].iloc[0]
            deltaX = x1-x2
            deltaY = y1-y2
            distance = math.sqrt((deltaX**2) + (deltaY**2))
            
            amplitudeLeftEyeDict[key] = (180/np.pi) * np.arctan(distance/(28*92))
            
            

            x1 = row1['Gaze point right X [DACS px]'].iloc[0]
            y1 = row1['Gaze point right Y [DACS px]'].iloc[0]
            x2 = row2.loc[:,'Gaze point right X [DACS px]'].iloc[0]
            y2 = row2.loc[:,'Gaze point right Y [DACS px]'].iloc[0]
            deltaX = x1-x2
            deltaY = y1-y2
            distance = math.sqrt((deltaX**2) + (deltaY**2))
            amplitudeRightEyeDict[key] = (180/np.pi) * np.arctan(distance/(28*92))




    print("Secondary Saccade Amplitude for Both Eyes (Mean and STD)")
    values = list(amplitudeBothEyesDict.values())
    print(np.mean(values), np.std(values))
    print(" ")
    print("Secondary Saccade Amplitude for Left Eyes (Mean and STD)")
    values = list(amplitudeLeftEyeDict.values())
    print(np.mean(values), np.std(values))
    print(" ")
    print("Secondary Saccade Amplitude for Right Eyes (Mean and STD)")
    values = list(amplitudeRightEyeDict.values())
    print(np.mean(values), np.std(values))
    print(" ")
   
    return amplitudeBothEyesDict, amplitudeRightEyeDict, amplitudeLeftEyeDict

### Finding Saccade Latency - using given timestamps

In [16]:
def saccadeLatency(primarySaccadeTime, times_dict):
    saccadeLatency =  defaultdict(int)
    i = 0
    for key,value in primarySaccadeTime.items(): 
        saccadeLatency[key] = abs(times_dict[key] - value)
    
    print("Primary Saccade Latency (Mean and STD)")
    values = list(saccadeLatency.values())
    print(np.mean(values), np.std(values))
    print(" ")
    return saccadeLatency



### Accuracy between primary saccade and stimuli: 

In [17]:
def accuracyPrimarySaccade(primarySaccadeIndex, dfSaccade):
    df_reset = dfSaccade.reset_index()
    df_reset['index'] = df_reset['index'].astype(int)
    stimuli_location = {
        "AOI hit [slothStimulus-final (1) - sloth1]" : [816, 396],
        "AOI hit [slothStimulus-final (1) - sloth2]" : [1588, 20],
        "AOI hit [slothStimulus-final (1) - sloth3]" : [51, 23],
        "AOI hit [slothStimulus-final (1) - sloth4]": [1588, 770],
        "AOI hit [slothStimulus-final (1) - sloth5]": [51, 767],
        "AOI hit [slothStimulus-final (1) - sloth6]" : [837, 287],
        "AOI hit [slothStimulus-final (1) - sloth7]": [1588, 395],
        "AOI hit [slothStimulus-final (1) - sloth8]" : [819, 770],
        "AOI hit [slothStimulus-final (1) - sloth9]" : [51, 395],
        "AOI hit [slothStimulus-final (1) - sloth10]" : [1588, 395],
        "AOI hit [slothStimulus-final (1) - sloth11]" : [1588, 395],
        "AOI hit [catStimulus-final (1) - cat1]" : [825, 377], 
        "AOI hit [catStimulus-final (1) - cat2]" : [1617, 5], 
        "AOI hit [catStimulus-final (1) - cat3]" : [1617, 325], 
        "AOI hit [catStimulus-final (1) - cat4]" : [13, 377], 
        "AOI hit [catStimulus-final (1) - cat5]" : [825, 5], 
        "AOI hit [catStimulus-final (1) - cat6]": [1617, 371], 
        "AOI hit [catStimulus-final (1) - cat7]": [13, 737], 
        "AOI hit [catStimulus-final (1) - cat8]" : [815, 749], 
        "AOI hit [catStimulus-final (1) - cat9]" : [13, 5], 
        "AOI hit [catStimulus-final (1) - cat10]" : [1617, 371],
       "AOI hit [catStimulus-final (1) - cat11]" : [825, 377]}
    #}

    #convert pixel values into inches
#     for key,value in stimuli_location.items(): 
#         stimuli_location[key] = np.array(value) * np.array([1/92, 1/92])


    accuracy_dist = defaultdict(int)

    for key,value in primarySaccadeIndex.items():
        if np.isnan(value)==False:
            
            row1 = df_reset[df_reset['index'] == value]
            x1 = row1['Gaze point X [DACS px]'].iloc[0]
            y1 = row1['Gaze point Y [DACS px]'].iloc[0]
            x2 = stimuli_location[key][0]
            y2 = stimuli_location[key][1]
            deltaX = x1-x2
            deltaY = y1-y2
            distance = math.sqrt((deltaX**2) + (deltaY**2))
            accuracy_dist[key] = distance/92
    
    print("Distance between primary saccade & latency (Mean and STD)")
    values = list(accuracy_dist.values())
    print(np.mean(values), np.std(values))
    print(" ")

    return accuracy_dist
    

### Accuracy between secondary saccade (time constraint) and stimuli: 

In [18]:
def accuracySecondarySaccadeTime(secondarySaccadesTime, dfSaccade):
    df_reset = dfSaccade.reset_index()
    df_reset['index'] = df_reset['index'].astype(int)
    stimuli_location = {
        "AOI hit [slothStimulus-final (1) - sloth1]" : [816, 396],
        "AOI hit [slothStimulus-final (1) - sloth2]" : [1588, 20],
        "AOI hit [slothStimulus-final (1) - sloth3]" : [51, 23],
        "AOI hit [slothStimulus-final (1) - sloth4]": [1588, 770],
        "AOI hit [slothStimulus-final (1) - sloth5]": [51, 767],
        "AOI hit [slothStimulus-final (1) - sloth6]" : [837, 287],
        "AOI hit [slothStimulus-final (1) - sloth7]": [1588, 395],
        "AOI hit [slothStimulus-final (1) - sloth8]" : [819, 770],
        "AOI hit [slothStimulus-final (1) - sloth9]" : [51, 395],
        "AOI hit [slothStimulus-final (1) - sloth10]" : [1588, 395],
        "AOI hit [slothStimulus-final (1) - sloth11]" : [1588, 395],
        "AOI hit [catStimulus-final (1) - cat1]" : [825, 377], 
        "AOI hit [catStimulus-final (1) - cat2]" : [1617, 5], 
        "AOI hit [catStimulus-final (1) - cat3]" : [1617, 325], 
        "AOI hit [catStimulus-final (1) - cat4]" : [13, 377], 
        "AOI hit [catStimulus-final (1) - cat5]" : [825, 5], 
        "AOI hit [catStimulus-final (1) - cat6]": [1617, 371], 
        "AOI hit [catStimulus-final (1) - cat7]": [13, 737], 
        "AOI hit [catStimulus-final (1) - cat8]" : [815, 749], 
        "AOI hit [catStimulus-final (1) - cat9]" : [13, 5], 
        "AOI hit [catStimulus-final (1) - cat10]" : [1617, 371],
       "AOI hit [catStimulus-final (1) - cat11]" : [825, 377]}
    # }

    #convert pixel values into inches
#     for key,value in stimuli_location.items(): 
#         stimuli_location[key] = np.array(value) * np.array([1/92, 1/92])


    accuracy_dist = defaultdict(int)

    for key,value in secondarySaccadesTime.items():
        if np.isnan(value)==False:
            row1 = df_reset[df_reset['index'] == value]
            x1 = row1['Gaze point X [DACS px]'].iloc[0]
            y1 = row1['Gaze point Y [DACS px]'].iloc[0]
            x2 = stimuli_location[key][0]
            y2 = stimuli_location[key][1]
            deltaX = x1-x2
            deltaY = y1-y2
            distance = math.sqrt((deltaX**2) + (deltaY**2))
            accuracy_dist[key] = distance/92


    
    print("Distance between secondary saccade (time only) & latency (Mean and STD)")
    values = list(accuracy_dist.values())
    print(np.mean(values), np.std(values))
    print(" ")
    return accuracy_dist

### Accuracy between secondary saccade (time and AOI constraint) and stimuli: 

In [19]:
def accuracySecondarySaccadeTimeandAOI(secondarySaccadesTimeAndAOI, dfSaccade):
    df_reset = dfSaccade.reset_index()
    df_reset['index'] = df_reset['index'].astype(int)
    stimuli_location = {
        "AOI hit [slothStimulus-final (1) - sloth1]" : [816, 396],
        "AOI hit [slothStimulus-final (1) - sloth2]" : [1588, 20],
        "AOI hit [slothStimulus-final (1) - sloth3]" : [51, 23],
        "AOI hit [slothStimulus-final (1) - sloth4]": [1588, 770],
        "AOI hit [slothStimulus-final (1) - sloth5]": [51, 767],
        "AOI hit [slothStimulus-final (1) - sloth6]" : [837, 287],
        "AOI hit [slothStimulus-final (1) - sloth7]": [1588, 395],
        "AOI hit [slothStimulus-final (1) - sloth8]" : [819, 770],
        "AOI hit [slothStimulus-final (1) - sloth9]" : [51, 395],
        "AOI hit [slothStimulus-final (1) - sloth10]" : [1588, 395],
        "AOI hit [slothStimulus-final (1) - sloth11]" : [1588, 395],
        "AOI hit [catStimulus-final (1) - cat1]" : [825, 377], 
        "AOI hit [catStimulus-final (1) - cat2]" : [1617, 5], 
        "AOI hit [catStimulus-final (1) - cat3]" : [1617, 325], 
        "AOI hit [catStimulus-final (1) - cat4]" : [13, 377], 
        "AOI hit [catStimulus-final (1) - cat5]" : [825, 5], 
        "AOI hit [catStimulus-final (1) - cat6]": [1617, 371], 
        "AOI hit [catStimulus-final (1) - cat7]": [13, 737], 
        "AOI hit [catStimulus-final (1) - cat8]" : [815, 749], 
        "AOI hit [catStimulus-final (1) - cat9]" : [13, 5], 
        "AOI hit [catStimulus-final (1) - cat10]" : [1617, 371],
       "AOI hit [catStimulus-final (1) - cat11]" : [825, 377]}
    # }

    #convert pixel values into inches
#     for key,value in stimuli_location.items(): 
#         stimuli_location[key] = np.array(value) * np.array([1/92, 1/92])


    accuracy_dist = defaultdict(int)

    for key,value in secondarySaccadesTimeAndAOI.items():
        if np.isnan(value)==False:
            row1 = df_reset[df_reset['index'] == value]
            x1 = row1['Gaze point X [DACS px]'].iloc[0]
            y1 = row1['Gaze point Y [DACS px]'].iloc[0]
            x2 = stimuli_location[key][0]
            y2 = stimuli_location[key][1]
            print(x2,y2)
            print(x1,y1)
            deltaX = x1-x2
            deltaY = y1-y2
            distance = math.sqrt((deltaX**2) + (deltaY**2))
            print(distance/92)
            
            accuracy_dist[key] = distance/92

    print("Distance between secondary saccade (time + AOI only) & latency (Mean and STD)")
    values = list(accuracy_dist.values())
    print(np.mean(values), np.std(values))
    print(" ")
    return accuracy_dist

### Corrective saccades between primary saccade and secondary saccade (time + AOI) 

In [20]:
def correctiveSaccadeTimeandAOI(secondarySaccadesTimeAndAOI, primarySaccadeIndex, dfSaccade):
    df_reset = dfSaccade.reset_index()
    df_reset['index'] = df_reset['index'].astype(int)
    correctiveSaccade = defaultdict(int)

    for key,value in secondarySaccadesTimeAndAOI.items():

        index_primary = df_reset[df_reset['index'] == primarySaccadeIndex[key]].index[0]
        index_secondary = df_reset[df_reset['index'] == value].index[0]
        correctiveSaccade[key] = int(index_secondary) - int(index_primary)

    print("Number of corrective saccade between secondary saccade (time + AOI only) & primary (Mean and STD)")
    values = list(correctiveSaccade.values())
    print(np.mean(values), np.std(values))
    print(" ")
    return correctiveSaccade

### Corrective saccades between primary saccade and secondary saccade (time) 

In [21]:
def correctiveSaccadeTime(secondarySaccadesTime, primarySaccadeIndex, dfSaccade):
    df_reset = dfSaccade.reset_index()
    df_reset['index'] = df_reset['index'].astype(int)
    correctiveSaccade = defaultdict(int)

    for key,value in secondarySaccadesTime.items():

        index_primary = df_reset[df_reset['index'] == primarySaccadeIndex[key]].index
        if len(index_primary) != 0:
            index_secondary = df_reset[df_reset['index'] == value].index[0]

            correctiveSaccade[key] = int(index_secondary) - int(index_primary[0])

    print("Number of corrective saccade between secondary saccade (time only) & primary (Mean and STD)")
    values = list(correctiveSaccade.values())
    print(np.mean(values), np.std(values))
    print(" ")
    return correctiveSaccade

## MAIN LOOP

In [76]:
import os
import glob

times_dict = {'AOI hit [slothStimulus-final (1) - sloth1]' : 0,
           'AOI hit [slothStimulus-final (1) - sloth2]': 2.5,
           'AOI hit [slothStimulus-final (1) - sloth3]': 5,
           'AOI hit [slothStimulus-final (1) - sloth4]': 7.5,
           'AOI hit [slothStimulus-final (1) - sloth5]': 10,
           'AOI hit [slothStimulus-final (1) - sloth6]': 12.5,
           'AOI hit [slothStimulus-final (1) - sloth7]': 15,
           'AOI hit [slothStimulus-final (1) - sloth8]': 17.5,
           'AOI hit [slothStimulus-final (1) - sloth9]': 20,
           'AOI hit [slothStimulus-final (1) - sloth10]': 22.5,
           'AOI hit [slothStimulus-final (1) - sloth11]': 25,
           'AOI hit [catStimulus-final (1) - cat1]': 0,
           'AOI hit [catStimulus-final (1) - cat2]': 2.5,
           'AOI hit [catStimulus-final (1) - cat3]': 5,
           'AOI hit [catStimulus-final (1) - cat4]': 7.5,
           'AOI hit [catStimulus-final (1) - cat5]': 10,
           'AOI hit [catStimulus-final (1) - cat6]': 12.5,
           'AOI hit [catStimulus-final (1) - cat7]': 15,
           'AOI hit [catStimulus-final (1) - cat8]': 17.5,
           'AOI hit [catStimulus-final (1) - cat9]': 20,
           'AOI hit [catStimulus-final (1) - cat10]': 22.5,
           'AOI hit [catStimulus-final (1) - cat11]': 25}

# plusLenseRight175 = defaultdict(int)
# plusLenseLeft175 = defaultdict(int)
# normal = defaultdict(int)
#dataCollection = [normal, plusLenseLeft175, plusLenseRight175]
# dataCollection = defaultdict(int) 

amplitudeBothEyesDictPrimaryRIGHT = []
amplitudeBothEyesDictSecondaryRIGHT = []
amplitudeRightEyesDictPrimaryRIGHT = []
amplitudeRightEyesDictSecondaryRIGHT = []
amplitudeLeftEyesDictPrimaryRIGHT = []
amplitudeLeftEyesDictSecondaryRIGHT = []
saccade_latencyRIGHT = []
accuracy_distPrimaryRIGHT = []
accuracy_distSecondaryRIGHT = []
correctiveSaccadeRIGHT = []

amplitudeBothEyesDictPrimaryLEFT = []
amplitudeBothEyesDictSecondaryLEFT = []
amplitudeRightEyesDictPrimaryLEFT = []
amplitudeRightEyesDictSecondaryLEFT = []
amplitudeLeftEyesDictPrimaryLEFT= []
amplitudeLeftEyesDictSecondaryLEFT= []
saccade_latencyLEFT = []
accuracy_distPrimaryLEFT = []
accuracy_distSecondaryLEFT = []
correctiveSaccadeLEFT = []

amplitudeBothEyesDictPrimaryNormal = []
amplitudeBothEyesDictSecondaryNormal = []
amplitudeRightEyesDictPrimaryNormal = []
amplitudeRightEyesDictSecondaryNormal = []
amplitudeLeftEyesDictPrimaryNormal= []
amplitudeLeftEyesDictSecondaryNormal= []
saccade_latencyNormal = []
accuracy_distPrimaryNormal = []
accuracy_distSecondaryNormal = []
correctiveSaccadeNormal = []

# folder_path = 'plusLenses'
# excel_files = glob.glob(os.path.join(folder_path, '*.xlsx'))
# print(excel_files)
folder_names = ['11-29-data/Normal', '11-29-data/Right', '11-29-data/Left']

files = glob.glob(os.path.join(folder_names[1], '*.xlsx'))
for idx, file_path in enumerate(files):
    print(file_path)
    dataCollection = defaultdict(int) 
    dataExport = pd.read_excel(file_path)
    dfSaccade, dataExportWithoutCalibration = preprocessing(dataExport)
    primarySaccadeTime, primarySaccadeIndex,secondarySaccadesTime, secondarySaccadesTimeAndAOI = primaryAndSecondarySaccade(dfSaccade, times_dict)
    dataCollection['amplitudeBothEyesDictPrimary'], dataCollection['amplitudeRightEyeDictPrimary'], dataCollection['amplitudeLeftEyeDictPrimary'] = amplitudePrimarySaccade(dfSaccade, primarySaccadeIndex, dataExportWithoutCalibration)
    dataCollection['amplitudeBothEyesDictSecondary'], dataCollection['amplitudeRightEyeDictSecondary'], dataCollection['amplitudeLeftEyeDictSecondary'] = secondarySaccadeTimeandAOIAmplitude(dfSaccade,dataExportWithoutCalibration, secondarySaccadesTimeAndAOI)
    dataCollection['saccade_latency'] = saccadeLatency(primarySaccadeTime, times_dict)
#     dataCollection['accuracy_distPrimary'] = accuracyPrimarySaccade(primarySaccadeIndex, dfSaccade)
#     dataCollection['accuracy_distSecondary'] = accuracySecondarySaccadeTimeandAOI(secondarySaccadesTimeAndAOI, dfSaccade)
    dataCollection['correctiveSaccade'] = correctiveSaccadeTimeandAOI(secondarySaccadesTimeAndAOI, primarySaccadeIndex, dfSaccade)
    amplitudeBothEyesDictPrimaryRIGHT.append(dataCollection['amplitudeBothEyesDictPrimary'])
    amplitudeBothEyesDictSecondaryRIGHT.append(dataCollection['amplitudeBothEyesDictSecondary'])
#     amplitudeRightEyesDictPrimaryRIGHT.append(dataCollection['amplitudeRightEyeDictPrimary'])
#     amplitudeRightEyesDictSecondaryRIGHT.append(dataCollection['amplitudeRightEyeDictSecondary'])
#     amplitudeLeftEyesDictPrimaryRIGHT.append(dataCollection['amplitudeLeftEyeDictPrimary'])
#     amplitudeLeftEyesDictSecondaryRIGHT.append(dataCollection['amplitudeLeftEyeDictSecondary'])
    saccade_latencyRIGHT.append(dataCollection['saccade_latency'])
#     accuracy_distPrimaryRIGHT.append(dataCollection['accuracy_distPrimary'])
#     accuracy_distSecondaryRIGHT.append(dataCollection['accuracy_distSecondary'])
    correctiveSaccadeRIGHT.append(dataCollection['correctiveSaccade'])


    print(" ")
    
#TRY THE MEAN (add to array) - to wash out the noise! - only 5 values for each of the features 


11-29-data/Right/YeRightEye.xlsx
LOOK HERE
Eyetracker timestamp [μs]                         6
Event                                          7023
Event value                                    7025
Gaze point X [MCS norm]                          15
Gaze point Y [MCS norm]                          15
Eye movement type                                 1
Presented Stimulus name                          17
AOI hit [slothStimulus-final (1) - sloth1]     3498
AOI hit [slothStimulus-final (1) - sloth2]     3498
AOI hit [slothStimulus-final (1) - sloth3]     3498
AOI hit [slothStimulus-final (1) - sloth4]     3498
AOI hit [slothStimulus-final (1) - sloth5]     3498
AOI hit [slothStimulus-final (1) - sloth6]     3498
AOI hit [slothStimulus-final (1) - sloth7]     3498
AOI hit [slothStimulus-final (1) - sloth8]     3498
AOI hit [slothStimulus-final (1) - sloth9]     3498
AOI hit [slothStimulus-final (1) - sloth10]    3498
AOI hit [slothStimulus-final (1) - sloth11]    3498
AOI hit [catStimulus-

defaultdict(int,
            {'AOI hit [slothStimulus-final (1) - sloth1]': 7.3770785642231855,
             'AOI hit [slothStimulus-final (1) - sloth2]': 17.623439111669857,
             'AOI hit [slothStimulus-final (1) - sloth3]': 26.732766935377228,
             'AOI hit [slothStimulus-final (1) - sloth4]': 1.1041586223874205,
             'AOI hit [slothStimulus-final (1) - sloth5]': 14.718740004802573,
             'AOI hit [slothStimulus-final (1) - sloth6]': 17.98680377899466,
             'AOI hit [slothStimulus-final (1) - sloth7]': 0.16192486822205523,
             'AOI hit [slothStimulus-final (1) - sloth8]': 0.3941256933156916,
             'AOI hit [slothStimulus-final (1) - sloth9]': 18.577340230437688,
             'AOI hit [slothStimulus-final (1) - sloth10]': 17.913644434455392,
             'AOI hit [slothStimulus-final (1) - sloth11]': 13.168360495960966,
             'AOI hit [catStimulus-final (1) - cat1]': 20.871093590611636,
             'AOI hit [catStimulus-fi

RIGHT


defaultdict(int,
            {'AOI hit [slothStimulus-final (1) - sloth1]': nan,
             'AOI hit [slothStimulus-final (1) - sloth2]': 66.5531885568354,
             'AOI hit [slothStimulus-final (1) - sloth3]': 25.775288049733515,
             'AOI hit [slothStimulus-final (1) - sloth4]': 2.272842615582537,
             'AOI hit [slothStimulus-final (1) - sloth5]': 15.24646209009169,
             'AOI hit [slothStimulus-final (1) - sloth6]': 17.176497810480733,
             'AOI hit [slothStimulus-final (1) - sloth7]': 3.276083748868185,
             'AOI hit [slothStimulus-final (1) - sloth8]': 0.6865978432295927,
             'AOI hit [slothStimulus-final (1) - sloth9]': 16.19602556842324,
             'AOI hit [slothStimulus-final (1) - sloth10]': 16.265649515334776,
             'AOI hit [slothStimulus-final (1) - sloth11]': 12.806384906278542,
             'AOI hit [catStimulus-final (1) - cat1]': 17.12345685097372,
             'AOI hit [catStimulus-final (1) - cat2]': 3.73

 
LEFT


...

RIGHT


defaultdict(int,
            {'AOI hit [slothStimulus-final (1) - sloth1]': 0.1424189592068876,
             'AOI hit [slothStimulus-final (1) - sloth2]': 24.83164125204702,
             'AOI hit [slothStimulus-final (1) - sloth3]': 28.281766477650482,
             'AOI hit [slothStimulus-final (1) - sloth4]': 32.181725637932765,
             'AOI hit [slothStimulus-final (1) - sloth5]': 30.57213966564292,
             'AOI hit [slothStimulus-final (1) - sloth6]': 23.312803239017118,
             'AOI hit [slothStimulus-final (1) - sloth7]': 20.040614803479375,
             'AOI hit [slothStimulus-final (1) - sloth8]': 16.80310673125459,
             'AOI hit [slothStimulus-final (1) - sloth9]': 19.336846223931364,
             'AOI hit [slothStimulus-final (1) - sloth10]': 30.355165689757783,
             'AOI hit [slothStimulus-final (1) - sloth11]': 15.21950000481651,
             'AOI hit [catStimulus-final (1) - cat1]': 0.44705261842372923,
             'AOI hit [catStimulus-final

 
LEFT


defaultdict(int,
            {'AOI hit [slothStimulus-final (1) - sloth1]': 0.9876139456655908,
             'AOI hit [slothStimulus-final (1) - sloth2]': 19.334707033817306,
             'AOI hit [slothStimulus-final (1) - sloth3]': 29.774288941632513,
             'AOI hit [slothStimulus-final (1) - sloth4]': 22.222126139791932,
             'AOI hit [slothStimulus-final (1) - sloth5]': 32.367973857656786,
             'AOI hit [slothStimulus-final (1) - sloth6]': 30.7689450541541,
             'AOI hit [slothStimulus-final (1) - sloth7]': 20.85527831942766,
             'AOI hit [slothStimulus-final (1) - sloth8]': 18.84713003556949,
             'AOI hit [slothStimulus-final (1) - sloth9]': 19.68060154969141,
             'AOI hit [slothStimulus-final (1) - sloth10]': 31.4129867034105,
             'AOI hit [slothStimulus-final (1) - sloth11]': 16.704526279012214,
             'AOI hit [catStimulus-final (1) - cat1]': 1.7104032932392672,
             'AOI hit [catStimulus-final (1)

Secondary Saccade Amplitude for Both Eyes (Mean and STD)
1.2374911962725523 0.6077483004885407
 
Secondary Saccade Amplitude for Left Eyes (Mean and STD)
1.2433889666087636 0.820184971487269
 
Secondary Saccade Amplitude for Right Eyes (Mean and STD)
1.1840570958873307 0.8554217720945535
 
Primary Saccade Latency (Mean and STD)
0.42389859090909066 0.1641915015643896
 
Number of corrective saccade between secondary saccade (time + AOI only) & primary (Mean and STD)
17.166666666666668 10.172129679778086
 
 


In [77]:
files = glob.glob(os.path.join(folder_names[0], '*.xlsx'))
for idx, file_path in enumerate(files):
    print(file_path)
    dataCollection = defaultdict(int) 
    dataExport = pd.read_excel(file_path)
    dfSaccade, dataExportWithoutCalibration = preprocessing(dataExport)
    primarySaccadeTime, primarySaccadeIndex,secondarySaccadesTime, secondarySaccadesTimeAndAOI = primaryAndSecondarySaccade(dfSaccade, times_dict)
    dataCollection['amplitudeBothEyesDictPrimary'], dataCollection['amplitudeRightEyeDictPrimary'], dataCollection['amplitudeLeftEyeDictPrimary'] = amplitudePrimarySaccade(dfSaccade, primarySaccadeIndex, dataExportWithoutCalibration)
    dataCollection['amplitudeBothEyesDictSecondary'], dataCollection['amplitudeRightEyeDictSecondary'], dataCollection['amplitudeLeftEyeDictSecondary'] = secondarySaccadeTimeandAOIAmplitude(dfSaccade,dataExportWithoutCalibration, secondarySaccadesTimeAndAOI)
    dataCollection['saccade_latency'] = saccadeLatency(primarySaccadeTime, times_dict)
#     dataCollection['accuracy_distPrimary'] = accuracyPrimarySaccade(primarySaccadeIndex, dfSaccade)
#     dataCollection['accuracy_distSecondary'] = accuracySecondarySaccadeTimeandAOI(secondarySaccadesTimeAndAOI, dfSaccade)
    dataCollection['correctiveSaccade'] = correctiveSaccadeTimeandAOI(secondarySaccadesTimeAndAOI, primarySaccadeIndex, dfSaccade)
    amplitudeBothEyesDictPrimaryNormal.append(dataCollection['amplitudeBothEyesDictPrimary'])
    amplitudeBothEyesDictSecondaryNormal.append(dataCollection['amplitudeBothEyesDictSecondary'])
#     amplitudeRightEyesDictPrimaryNormal.append(dataCollection['amplitudeRightEyeDictPrimary'])
#     amplitudeRightEyesDictSecondaryNormal.append(dataCollection['amplitudeRightEyeDictSecondary'])
#     amplitudeLeftEyesDictPrimaryNormal.append(dataCollection['amplitudeLeftEyeDictPrimary'])
#     amplitudeLeftEyesDictSecondaryNormal.append(dataCollection['amplitudeLeftEyeDictSecondary'])
    saccade_latencyNormal.append(dataCollection['saccade_latency'])
#     accuracy_distPrimaryNormal.append(dataCollection['accuracy_distPrimary'])
#     accuracy_distSecondaryNormal.append(dataCollection['accuracy_distSecondary'])
    correctiveSaccadeNormal.append(dataCollection['correctiveSaccade'])


    print(" ")

11-29-data/Normal/SanaNormal.xlsx
LOOK HERE
Eyetracker timestamp [μs]                        16
Event                                          7091
Event value                                    7092
Gaze point X [MCS norm]                          13
Gaze point Y [MCS norm]                          13
Eye movement type                                 1
Presented Stimulus name                          15
AOI hit [slothStimulus-final (1) - sloth1]     3555
AOI hit [slothStimulus-final (1) - sloth2]     3555
AOI hit [slothStimulus-final (1) - sloth3]     3555
AOI hit [slothStimulus-final (1) - sloth4]     3555
AOI hit [slothStimulus-final (1) - sloth5]     3555
AOI hit [slothStimulus-final (1) - sloth6]     3555
AOI hit [slothStimulus-final (1) - sloth7]     3555
AOI hit [slothStimulus-final (1) - sloth8]     3555
AOI hit [slothStimulus-final (1) - sloth9]     3555
AOI hit [slothStimulus-final (1) - sloth10]    3555
AOI hit [slothStimulus-final (1) - sloth11]    3555
AOI hit [catStimulus

defaultdict(int,
            {'AOI hit [slothStimulus-final (1) - sloth1]': 5.560777085494331,
             'AOI hit [slothStimulus-final (1) - sloth2]': 1.2431788818482843,
             'AOI hit [slothStimulus-final (1) - sloth3]': 27.832057776820204,
             'AOI hit [slothStimulus-final (1) - sloth4]': 14.639654915865016,
             'AOI hit [slothStimulus-final (1) - sloth5]': 28.91665628090242,
             'AOI hit [slothStimulus-final (1) - sloth6]': 21.434821883128283,
             'AOI hit [slothStimulus-final (1) - sloth7]': 16.93574149215741,
             'AOI hit [slothStimulus-final (1) - sloth8]': 17.980306424606205,
             'AOI hit [slothStimulus-final (1) - sloth9]': 16.550537535754113,
             'AOI hit [slothStimulus-final (1) - sloth10]': 1.1547232560964675,
             'AOI hit [slothStimulus-final (1) - sloth11]': 15.361741937747208,
             'AOI hit [catStimulus-final (1) - cat1]': 8.326296967213171,
             'AOI hit [catStimulus-final 

RIGHT


defaultdict(int,
            {'AOI hit [slothStimulus-final (1) - sloth1]': 4.520261296146526,
             'AOI hit [slothStimulus-final (1) - sloth2]': 1.4164159031939683,
             'AOI hit [slothStimulus-final (1) - sloth3]': 28.30502283899657,
             'AOI hit [slothStimulus-final (1) - sloth4]': 14.547428662758412,
             'AOI hit [slothStimulus-final (1) - sloth5]': 28.745862597947255,
             'AOI hit [slothStimulus-final (1) - sloth6]': 21.602294840759207,
             'AOI hit [slothStimulus-final (1) - sloth7]': 17.423074821979146,
             'AOI hit [slothStimulus-final (1) - sloth8]': 17.96896499379893,
             'AOI hit [slothStimulus-final (1) - sloth9]': 16.789998737658888,
             'AOI hit [slothStimulus-final (1) - sloth10]': 1.04904289873689,
             'AOI hit [slothStimulus-final (1) - sloth11]': 15.516277477194388,
             'AOI hit [catStimulus-final (1) - cat1]': 8.550109265246382,
             'AOI hit [catStimulus-final (1

 
LEFT


...

RIGHT


defaultdict(int,
            {'AOI hit [slothStimulus-final (1) - sloth1]': 3.273585992276314,
             'AOI hit [slothStimulus-final (1) - sloth2]': 18.162944329099492,
             'AOI hit [slothStimulus-final (1) - sloth3]': 16.285324997471875,
             'AOI hit [slothStimulus-final (1) - sloth4]': 0.648436860126655,
             'AOI hit [slothStimulus-final (1) - sloth5]': 7.164215139017842,
             'AOI hit [slothStimulus-final (1) - sloth6]': 10.88940321786496,
             'AOI hit [slothStimulus-final (1) - sloth7]': 4.710253147640221,
             'AOI hit [slothStimulus-final (1) - sloth8]': 16.780605635226053,
             'AOI hit [slothStimulus-final (1) - sloth9]': 3.369080747881437,
             'AOI hit [slothStimulus-final (1) - sloth10]': 9.959823620616524,
             'AOI hit [slothStimulus-final (1) - sloth11]': 8.957597756407822,
             'AOI hit [catStimulus-final (1) - cat1]': 3.70307604669496,
             'AOI hit [catStimulus-final (1) - 

 
LEFT


defaultdict(int,
            {'AOI hit [slothStimulus-final (1) - sloth1]': 2.9229597663858202,
             'AOI hit [slothStimulus-final (1) - sloth2]': 17.61405415411736,
             'AOI hit [slothStimulus-final (1) - sloth3]': 16.05819231766352,
             'AOI hit [slothStimulus-final (1) - sloth4]': 10.571897516540693,
             'AOI hit [slothStimulus-final (1) - sloth5]': 7.670334547241893,
             'AOI hit [slothStimulus-final (1) - sloth6]': 11.07037259330098,
             'AOI hit [slothStimulus-final (1) - sloth7]': 3.9526330119786173,
             'AOI hit [slothStimulus-final (1) - sloth8]': 26.666893709558558,
             'AOI hit [slothStimulus-final (1) - sloth9]': 2.974285861581482,
             'AOI hit [slothStimulus-final (1) - sloth10]': 8.939940832358394,
             'AOI hit [slothStimulus-final (1) - sloth11]': 8.707620839195107,
             'AOI hit [catStimulus-final (1) - cat1]': 4.1746534643722075,
             'AOI hit [catStimulus-final (1)

Secondary Saccade Amplitude for Both Eyes (Mean and STD)
7.200656321063188 6.515561099727023
 
Secondary Saccade Amplitude for Left Eyes (Mean and STD)
8.971150285769909 9.502544278549134
 
Secondary Saccade Amplitude for Right Eyes (Mean and STD)
7.316749243643958 6.593732625590673
 
Primary Saccade Latency (Mean and STD)
0.20798499999999998 0.16103427354928354
 
Number of corrective saccade between secondary saccade (time + AOI only) & primary (Mean and STD)
11.777777777777779 6.032831982443656
 
 


In [78]:
files = glob.glob(os.path.join(folder_names[2], '*.xlsx'))
for idx, file_path in enumerate(files):
    print(file_path)
    dataCollection = defaultdict(int) 
    dataExport = pd.read_excel(file_path)
    dfSaccade, dataExportWithoutCalibration = preprocessing(dataExport)
    primarySaccadeTime, primarySaccadeIndex,secondarySaccadesTime, secondarySaccadesTimeAndAOI = primaryAndSecondarySaccade(dfSaccade, times_dict)
    dataCollection['amplitudeBothEyesDictPrimary'], dataCollection['amplitudeRightEyeDictPrimary'], dataCollection['amplitudeLeftEyeDictPrimary'] = amplitudePrimarySaccade(dfSaccade, primarySaccadeIndex, dataExportWithoutCalibration)
    dataCollection['amplitudeBothEyesDictSecondary'], dataCollection['amplitudeRightEyeDictSecondary'], dataCollection['amplitudeLeftEyeDictSecondary'] = secondarySaccadeTimeandAOIAmplitude(dfSaccade,dataExportWithoutCalibration, secondarySaccadesTimeAndAOI)
    dataCollection['saccade_latency'] = saccadeLatency(primarySaccadeTime, times_dict)
#     dataCollection['accuracy_distPrimary'] = accuracyPrimarySaccade(primarySaccadeIndex, dfSaccade)
#     dataCollection['accuracy_distSecondary'] = accuracySecondarySaccadeTimeandAOI(secondarySaccadesTimeAndAOI, dfSaccade)
    dataCollection['correctiveSaccade'] = correctiveSaccadeTimeandAOI(secondarySaccadesTimeAndAOI, primarySaccadeIndex, dfSaccade)
    amplitudeBothEyesDictPrimaryLEFT.append(dataCollection['amplitudeBothEyesDictPrimary'])
    amplitudeBothEyesDictSecondaryLEFT.append(dataCollection['amplitudeBothEyesDictSecondary'])
#     amplitudeRightEyesDictPrimaryLEFT.append(dataCollection['amplitudeRightEyeDictPrimary'])
#     amplitudeRightEyesDictSecondaryLEFT.append(dataCollection['amplitudeRightEyeDictSecondary'])
#     amplitudeLeftEyesDictPrimaryLEFT.append(dataCollection['amplitudeLeftEyeDictPrimary'])
#     amplitudeLeftEyesDictSecondaryLEFT.append(dataCollection['amplitudeLeftEyeDictSecondary'])
    saccade_latencyLEFT.append(dataCollection['saccade_latency'])
#     accuracy_distPrimaryLEFT.append(dataCollection['accuracy_distPrimary'])
#     accuracy_distSecondaryLEFT.append(dataCollection['accuracy_distSecondary'])
    correctiveSaccadeLEFT.append(dataCollection['correctiveSaccade'])


    print(" ")

11-29-data/Left/YeLeftEye.xlsx
LOOK HERE
Eyetracker timestamp [μs]                         6
Event                                          6759
Event value                                    6761
Gaze point X [MCS norm]                          15
Gaze point Y [MCS norm]                          15
Eye movement type                                 1
Presented Stimulus name                          17
AOI hit [slothStimulus-final (1) - sloth1]     3425
AOI hit [slothStimulus-final (1) - sloth2]     3425
AOI hit [slothStimulus-final (1) - sloth3]     3425
AOI hit [slothStimulus-final (1) - sloth4]     3425
AOI hit [slothStimulus-final (1) - sloth5]     3425
AOI hit [slothStimulus-final (1) - sloth6]     3425
AOI hit [slothStimulus-final (1) - sloth7]     3425
AOI hit [slothStimulus-final (1) - sloth8]     3425
AOI hit [slothStimulus-final (1) - sloth9]     3425
AOI hit [slothStimulus-final (1) - sloth10]    3425
AOI hit [slothStimulus-final (1) - sloth11]    3425
AOI hit [catStimulus-fi

defaultdict(int,
            {'AOI hit [slothStimulus-final (1) - sloth1]': 7.905334633942032,
             'AOI hit [slothStimulus-final (1) - sloth2]': 17.381434778105046,
             'AOI hit [slothStimulus-final (1) - sloth3]': 29.4155353553435,
             'AOI hit [slothStimulus-final (1) - sloth4]': 19.747376896335798,
             'AOI hit [slothStimulus-final (1) - sloth5]': 10.173487499918533,
             'AOI hit [slothStimulus-final (1) - sloth6]': 6.801426237643962,
             'AOI hit [slothStimulus-final (1) - sloth7]': 7.7333782761743874,
             'AOI hit [slothStimulus-final (1) - sloth8]': 7.519674444095064,
             'AOI hit [slothStimulus-final (1) - sloth9]': 13.851052659517958,
             'AOI hit [slothStimulus-final (1) - sloth10]': 14.642191956445604,
             'AOI hit [slothStimulus-final (1) - sloth11]': 15.496671173867338,
             'AOI hit [catStimulus-final (1) - cat1]': 12.924695533198388,
             'AOI hit [catStimulus-final (

RIGHT


defaultdict(int,
            {'AOI hit [slothStimulus-final (1) - sloth1]': 7.905334633942032,
             'AOI hit [slothStimulus-final (1) - sloth2]': 17.381434778105046,
             'AOI hit [slothStimulus-final (1) - sloth3]': 29.4155353553435,
             'AOI hit [slothStimulus-final (1) - sloth4]': 19.747376896335798,
             'AOI hit [slothStimulus-final (1) - sloth5]': 10.173487499918533,
             'AOI hit [slothStimulus-final (1) - sloth6]': 6.801426237643962,
             'AOI hit [slothStimulus-final (1) - sloth7]': 7.7333782761743874,
             'AOI hit [slothStimulus-final (1) - sloth8]': 7.519674444095064,
             'AOI hit [slothStimulus-final (1) - sloth9]': 13.626266098572714,
             'AOI hit [slothStimulus-final (1) - sloth10]': 15.142021444747302,
             'AOI hit [slothStimulus-final (1) - sloth11]': 14.979760622292357,
             'AOI hit [catStimulus-final (1) - cat1]': 12.665120609744159,
             'AOI hit [catStimulus-final (

 
LEFT


...

RIGHT


defaultdict(int,
            {'AOI hit [slothStimulus-final (1) - sloth1]': 4.186462095407871,
             'AOI hit [slothStimulus-final (1) - sloth2]': 14.726808401637816,
             'AOI hit [slothStimulus-final (1) - sloth3]': 23.14441483094957,
             'AOI hit [slothStimulus-final (1) - sloth4]': 0.5613485352959448,
             'AOI hit [slothStimulus-final (1) - sloth5]': 24.99695080424625,
             'AOI hit [slothStimulus-final (1) - sloth6]': 17.030065503916447,
             'AOI hit [slothStimulus-final (1) - sloth7]': 17.790483740189096,
             'AOI hit [slothStimulus-final (1) - sloth8]': 18.360487331914033,
             'AOI hit [slothStimulus-final (1) - sloth9]': 8.418090887930274,
             'AOI hit [slothStimulus-final (1) - sloth10]': 15.047380023559109,
             'AOI hit [slothStimulus-final (1) - sloth11]': 2.6029146746410987,
             'AOI hit [catStimulus-final (1) - cat1]': 19.514064513746643,
             'AOI hit [catStimulus-final 

 
LEFT


defaultdict(int,
            {'AOI hit [slothStimulus-final (1) - sloth1]': nan,
             'AOI hit [slothStimulus-final (1) - sloth2]': nan,
             'AOI hit [slothStimulus-final (1) - sloth3]': nan,
             'AOI hit [slothStimulus-final (1) - sloth4]': nan,
             'AOI hit [slothStimulus-final (1) - sloth5]': nan,
             'AOI hit [slothStimulus-final (1) - sloth6]': nan,
             'AOI hit [slothStimulus-final (1) - sloth7]': nan,
             'AOI hit [slothStimulus-final (1) - sloth8]': nan,
             'AOI hit [slothStimulus-final (1) - sloth9]': nan,
             'AOI hit [slothStimulus-final (1) - sloth10]': nan,
             'AOI hit [slothStimulus-final (1) - sloth11]': nan,
             'AOI hit [catStimulus-final (1) - cat1]': nan,
             'AOI hit [catStimulus-final (1) - cat2]': nan,
             'AOI hit [catStimulus-final (1) - cat3]': 44.53730034327585,
             'AOI hit [catStimulus-final (1) - cat4]': 2.996149387242323,
         

Secondary Saccade Amplitude for Both Eyes (Mean and STD)
2.5958450637052253 3.556594602516792
 
Secondary Saccade Amplitude for Left Eyes (Mean and STD)
nan nan
 
Secondary Saccade Amplitude for Right Eyes (Mean and STD)
2.5000848556795465 3.615815241376124
 
Primary Saccade Latency (Mean and STD)
0.21555890909090902 0.16574490870115863
 
Number of corrective saccade between secondary saccade (time + AOI only) & primary (Mean and STD)
16.25 6.466258578188781
 
 


### new normal data


In [24]:
times_dict = {'AOI hit [slothStimulus-final (1) - sloth1]' : 0,
           'AOI hit [slothStimulus-final (1) - sloth2]': 2.5,
           'AOI hit [slothStimulus-final (1) - sloth3]': 5,
           'AOI hit [slothStimulus-final (1) - sloth4]': 7.5,
           'AOI hit [slothStimulus-final (1) - sloth5]': 10,
           'AOI hit [slothStimulus-final (1) - sloth6]': 12.5,
           'AOI hit [slothStimulus-final (1) - sloth7]': 15,
           'AOI hit [slothStimulus-final (1) - sloth8]': 17.5,
           'AOI hit [slothStimulus-final (1) - sloth9]': 20,
           'AOI hit [slothStimulus-final (1) - sloth10]': 22.5,
           'AOI hit [slothStimulus-final (1) - sloth11]': 25,
           'AOI hit [catStimulus-final (1) - cat1]': 0,
           'AOI hit [catStimulus-final (1) - cat2]': 2.5,
           'AOI hit [catStimulus-final (1) - cat3]': 5,
           'AOI hit [catStimulus-final (1) - cat4]': 7.5,
           'AOI hit [catStimulus-final (1) - cat5]': 10,
           'AOI hit [catStimulus-final (1) - cat6]': 12.5,
           'AOI hit [catStimulus-final (1) - cat7]': 15,
           'AOI hit [catStimulus-final (1) - cat8]': 17.5,
           'AOI hit [catStimulus-final (1) - cat9]': 20,
           'AOI hit [catStimulus-final (1) - cat10]': 22.5,
           'AOI hit [catStimulus-final (1) - cat11]': 25}

import os
folder_names = ['2-14 data Normal']

amplitudeBothEyesDictPrimaryNORMAL_new = []
amplitudeBothEyesDictSecondaryNORMAL_new = []

saccade_latencyNORMAL_new = []

correctiveSaccadeNORMAL_new = []

files = glob.glob(os.path.join(folder_names[0], '*.xlsx'))
for idx, file_path in enumerate(files):
    print(file_path)
    dataCollection = defaultdict(int) 
    dataExport = pd.read_excel(file_path)
    dfSaccade, dataExportWithoutCalibration = preprocessing(dataExport)
    primarySaccadeTime, primarySaccadeIndex,secondarySaccadesTime, secondarySaccadesTimeAndAOI = primaryAndSecondarySaccade(dfSaccade, times_dict)
    dataCollection['amplitudeBothEyesDictPrimary'], dataCollection['amplitudeRightEyeDictPrimary'], dataCollection['amplitudeLeftEyeDictPrimary'] = amplitudePrimarySaccade(dfSaccade, primarySaccadeIndex, dataExportWithoutCalibration)
    dataCollection['amplitudeBothEyesDictSecondary'], dataCollection['amplitudeRightEyeDictSecondary'], dataCollection['amplitudeLeftEyeDictSecondary'] = secondarySaccadeTimeandAOIAmplitude(dfSaccade,dataExportWithoutCalibration, secondarySaccadesTimeAndAOI)
    dataCollection['saccade_latency'] = saccadeLatency(primarySaccadeTime, times_dict)
    dataCollection['accuracy_distPrimary'] = accuracyPrimarySaccade(primarySaccadeIndex, dfSaccade)
    dataCollection['accuracy_distSecondary'] = accuracySecondarySaccadeTimeandAOI(secondarySaccadesTimeAndAOI, dfSaccade)
    dataCollection['correctiveSaccade'] = correctiveSaccadeTimeandAOI(secondarySaccadesTimeAndAOI, primarySaccadeIndex, dfSaccade)
    amplitudeBothEyesDictPrimaryNORMAL_new.append(dataCollection['amplitudeBothEyesDictPrimary'])
    amplitudeBothEyesDictSecondaryNORMAL_new.append(dataCollection['amplitudeBothEyesDictSecondary'])
#     amplitudeRightEyesDictPrimaryRIGHT.append(dataCollection['amplitudeRightEyeDictPrimary'])
#     amplitudeRightEyesDictSecondaryRIGHT.append(dataCollection['amplitudeRightEyeDictSecondary'])
#     amplitudeLeftEyesDictPrimaryRIGHT.append(dataCollection['amplitudeLeftEyeDictPrimary'])
#     amplitudeLeftEyesDictSecondaryRIGHT.append(dataCollection['amplitudeLeftEyeDictSecondary'])
    saccade_latencyNORMAL_new.append(dataCollection['saccade_latency'])
#     accuracy_distPrimaryNORMAL_new.append(dataCollection['accuracy_distPrimary'])
#     accuracy_distSecondaryRIGHT.append(dataCollection['accuracy_distSecondary'])
    correctiveSaccadeNORMAL_new.append(dataCollection['correctiveSaccade'])
    
    
    #for p - values
#     amplitudeBothEyesDictPrimaryNormal.append(dataCollection['amplitudeBothEyesDictPrimary'])
#     amplitudeBothEyesDictSecondaryNormal.append(dataCollection['amplitudeBothEyesDictSecondary'])
    
#     saccade_latencyNormal.append(dataCollection['saccade_latency'])
   
#     correctiveSaccadeNormal.append(dataCollection['correctiveSaccade'])


    print(" ")

2-14 data Normal/SteveData.xlsx
LOOK HERE
Eyetracker timestamp [μs]                         4
Event                                          6962
Event value                                    6963
Gaze point X [MCS norm]                           9
Gaze point Y [MCS norm]                           9
Eye movement type                                 1
Presented Stimulus name                          11
AOI hit [slothStimulus-final (1) - sloth1]     3462
AOI hit [slothStimulus-final (1) - sloth2]     3462
AOI hit [slothStimulus-final (1) - sloth3]     3462
AOI hit [slothStimulus-final (1) - sloth4]     3462
AOI hit [slothStimulus-final (1) - sloth5]     3462
AOI hit [slothStimulus-final (1) - sloth6]     3462
AOI hit [slothStimulus-final (1) - sloth7]     3462
AOI hit [slothStimulus-final (1) - sloth8]     3462
AOI hit [slothStimulus-final (1) - sloth9]     3462
AOI hit [slothStimulus-final (1) - sloth10]    3462
AOI hit [slothStimulus-final (1) - sloth11]    3462
AOI hit [catStimulus-f

defaultdict(int,
            {'AOI hit [slothStimulus-final (1) - sloth1]': 13.667422463310574,
             'AOI hit [slothStimulus-final (1) - sloth2]': 1.430998156002041,
             'AOI hit [slothStimulus-final (1) - sloth3]': 27.5264162368608,
             'AOI hit [slothStimulus-final (1) - sloth4]': 7.648104197220778,
             'AOI hit [slothStimulus-final (1) - sloth5]': 11.308976836523994,
             'AOI hit [slothStimulus-final (1) - sloth6]': 3.5389819434964522,
             'AOI hit [slothStimulus-final (1) - sloth7]': 15.755860287772961,
             'AOI hit [slothStimulus-final (1) - sloth8]': 8.73006098557328,
             'AOI hit [slothStimulus-final (1) - sloth9]': 7.821163916511113,
             'AOI hit [slothStimulus-final (1) - sloth10]': 5.213650541763496,
             'AOI hit [slothStimulus-final (1) - sloth11]': 5.313416707466987,
             'AOI hit [catStimulus-final (1) - cat1]': 1.6771452500840573,
             'AOI hit [catStimulus-final (1) -

RIGHT


defaultdict(int,
            {'AOI hit [slothStimulus-final (1) - sloth1]': 13.582078953123837,
             'AOI hit [slothStimulus-final (1) - sloth2]': 1.1976020170727537,
             'AOI hit [slothStimulus-final (1) - sloth3]': 27.629692457439578,
             'AOI hit [slothStimulus-final (1) - sloth4]': 7.578070329930867,
             'AOI hit [slothStimulus-final (1) - sloth5]': 11.41094869906117,
             'AOI hit [slothStimulus-final (1) - sloth6]': 3.3328549289675506,
             'AOI hit [slothStimulus-final (1) - sloth7]': 15.494487643144419,
             'AOI hit [slothStimulus-final (1) - sloth8]': 8.781100192811968,
             'AOI hit [slothStimulus-final (1) - sloth9]': 6.572850031100013,
             'AOI hit [slothStimulus-final (1) - sloth10]': 4.661860490775838,
             'AOI hit [slothStimulus-final (1) - sloth11]': 5.6284406150018045,
             'AOI hit [catStimulus-final (1) - cat1]': 1.006952507751769,
             'AOI hit [catStimulus-final (1

 
LEFT


...

RIGHT


defaultdict(int,
            {'AOI hit [slothStimulus-final (1) - sloth1]': 1.1175124985919072,
             'AOI hit [slothStimulus-final (1) - sloth2]': 18.640551633845817,
             'AOI hit [slothStimulus-final (1) - sloth3]': 20.641534311582046,
             'AOI hit [slothStimulus-final (1) - sloth4]': 20.78600795087554,
             'AOI hit [slothStimulus-final (1) - sloth5]': 13.797835695818844,
             'AOI hit [slothStimulus-final (1) - sloth6]': 2.978509039585756,
             'AOI hit [slothStimulus-final (1) - sloth7]': 12.85809017299818,
             'AOI hit [slothStimulus-final (1) - sloth8]': 12.89043800467801,
             'AOI hit [slothStimulus-final (1) - sloth9]': 12.496034736045958,
             'AOI hit [slothStimulus-final (1) - sloth10]': 18.492875325992053,
             'AOI hit [slothStimulus-final (1) - sloth11]': 14.627395567376055,
             'AOI hit [catStimulus-final (1) - cat1]': 0.85907017047145,
             'AOI hit [catStimulus-final (1

 
LEFT


defaultdict(int,
            {'AOI hit [slothStimulus-final (1) - sloth1]': 1.4342759292014289,
             'AOI hit [slothStimulus-final (1) - sloth2]': 18.278701708894125,
             'AOI hit [slothStimulus-final (1) - sloth3]': 21.473656293501424,
             'AOI hit [slothStimulus-final (1) - sloth4]': 19.544066663437487,
             'AOI hit [slothStimulus-final (1) - sloth5]': 13.039421576153876,
             'AOI hit [slothStimulus-final (1) - sloth6]': 2.0533609759933893,
             'AOI hit [slothStimulus-final (1) - sloth7]': 13.116750767966975,
             'AOI hit [slothStimulus-final (1) - sloth8]': 12.539380286248374,
             'AOI hit [slothStimulus-final (1) - sloth9]': 11.822051217081624,
             'AOI hit [slothStimulus-final (1) - sloth10]': 17.923737537629165,
             'AOI hit [slothStimulus-final (1) - sloth11]': 14.837699594837375,
             'AOI hit [catStimulus-final (1) - cat1]': 0.9383134620999503,
             'AOI hit [catStimulus-fi

Secondary Saccade Amplitude for Both Eyes (Mean and STD)
9.77263341431072 7.405366215631761
 
Secondary Saccade Amplitude for Left Eyes (Mean and STD)
9.935685974097614 7.316844589208239
 
Secondary Saccade Amplitude for Right Eyes (Mean and STD)
9.761372975951637 7.332524352652488
 
Primary Saccade Latency (Mean and STD)
0.10871577272727274 0.10161155968774221
 
Distance between primary saccade & latency (Mean and STD)
9.411916730638493 5.07754444442417
 
816 396
957.0 520.0
2.040961678883067
1588 20
1761.0 150.0
2.352174917624153
1588 770
1734.0 886.0
2.026875196548089
51 767
660.0 413.0
7.656657838950355
1588 395
1723.0 518.0
1.985117120246664
51 395
204.0 500.0
2.017000055764189
1588 395
884.0 561.0
7.862024972802355
825 377
971.0 522.0
2.2366226987152564
1617 5
1740.0 144.0
2.017468607964194
1617 325
1758.0 821.0
5.604913200476625
825 5
968.0 413.0
4.6992865363638625
1617 371
1702.0 517.0
1.8363132395644328
13 737
139.0 857.0
1.891304347826087
Distance between secondary saccade (t

In [44]:
print(amplitudeLeftEyesDictSecondaryNormal)

[defaultdict(<class 'int'>, {'AOI hit [slothStimulus-final (1) - sloth1]': 0.2224203870465026, 'AOI hit [slothStimulus-final (1) - sloth2]': 0.9293104212633397, 'AOI hit [slothStimulus-final (1) - sloth3]': 0.8798321072479919, 'AOI hit [slothStimulus-final (1) - sloth5]': 1.8281116314444037, 'AOI hit [catStimulus-final (1) - cat1]': 1.9230628433933346, 'AOI hit [catStimulus-final (1) - cat4]': 0.5379500410636265, 'AOI hit [catStimulus-final (1) - cat5]': 7.093146375118457, 'AOI hit [catStimulus-final (1) - cat6]': 0.9714586944681032, 'AOI hit [catStimulus-final (1) - cat7]': 0.915909792160312, 'AOI hit [catStimulus-final (1) - cat11]': 2.2227656124906336}), defaultdict(<class 'int'>, {'AOI hit [slothStimulus-final (1) - sloth5]': 18.448796414798625, 'AOI hit [catStimulus-final (1) - cat5]': 0.8303816665943003, 'AOI hit [catStimulus-final (1) - cat7]': 31.522062318997875, 'AOI hit [catStimulus-final (1) - cat9]': 0.6263196946701083}), defaultdict(<class 'int'>, {'AOI hit [slothStimulus-

In [45]:
print(amplitudeLeftEyesDictSecondaryRIGHT)

[defaultdict(<class 'int'>, {'AOI hit [slothStimulus-final (1) - sloth1]': 4.494450893227868, 'AOI hit [slothStimulus-final (1) - sloth2]': 2.7631375837457877, 'AOI hit [slothStimulus-final (1) - sloth3]': 0.5821127561346091, 'AOI hit [slothStimulus-final (1) - sloth4]': 1.1609154459803541, 'AOI hit [slothStimulus-final (1) - sloth5]': 1.0760350490198247, 'AOI hit [slothStimulus-final (1) - sloth6]': 1.1609154459803541, 'AOI hit [slothStimulus-final (1) - sloth7]': 0.11977747125629368, 'AOI hit [slothStimulus-final (1) - sloth8]': 0.16192486822205523, 'AOI hit [slothStimulus-final (1) - sloth9]': 19.651958540376697, 'AOI hit [slothStimulus-final (1) - sloth11]': 0.5429838366718922, 'AOI hit [catStimulus-final (1) - cat1]': 20.293295805403286, 'AOI hit [catStimulus-final (1) - cat2]': 0.4231780781047682, 'AOI hit [catStimulus-final (1) - cat3]': 0.6151638453769223, 'AOI hit [catStimulus-final (1) - cat4]': 3.5359208477377324, 'AOI hit [catStimulus-final (1) - cat5]': 0.0801951614900229,

## ML df creation and preprocessing

In [55]:
key_list = []
sample_list = []
for i, j, k, l in zip(amplitudeBothEyesDictPrimaryLEFT, amplitudeBothEyesDictSecondaryLEFT, saccade_latencyLEFT, correctiveSaccadeLEFT):
    #looping in order 
    #extracting samples 
    #in dictionary 
    print(len(j.keys()))
    for key in j.keys():
        key_list.append(key)
        sample_list.append([i[key], j[key], k[key], l[key]])
        
    
df_left = pd.DataFrame(sample_list)
df_left['normal'] = 0

key_list = []
sample_list = []
for i, j, k, l in zip(amplitudeBothEyesDictPrimaryRIGHT, amplitudeBothEyesDictSecondaryRIGHT, saccade_latencyRIGHT, correctiveSaccadeRIGHT):
    #looping in order 
    #extracting samples 
    #in dictionary 
    print(len(j.keys()))
    for key in j.keys():
        key_list.append(key)
        sample_list.append([i[key], j[key], k[key], l[key]])
        
    
df_right = pd.DataFrame(sample_list)
df_right['normal'] = 0


key_list = []
sample_list = []
for i, j, k, l in zip(amplitudeBothEyesDictPrimaryNormal, amplitudeBothEyesDictSecondaryNormal, saccade_latencyNormal, correctiveSaccadeNormal):
    #looping in order 
    #extracting samples 
    #in dictionary 
    print(len(j.keys()))
    for key in j.keys():
        key_list.append(key)
        sample_list.append([i[key], j[key], k[key], l[key]])
        

        
    
df_normal = pd.DataFrame(sample_list)
df_normal['normal'] = 1

result = pd.concat([df_left, df_right, df_normal], ignore_index=True)

display(result)
    
result.to_csv('TestDataCompiled.csv', index=False)

15
14
5
17
16
21
14
6
17
12
10
4
20
12
9


,0,1,2,3,normal
0,7.905335,0.907229,0.124998,7,0
1,17.381435,0.563108,0.341670,13,0
2,29.415535,1.097195,0.308346,16,0
3,10.173487,1.182019,0.175026,9,0
4,6.801426,7.733378,0.391700,28,0
5,7.733378,11.071244,0.075037,26,0
6,12.924696,1.971763,0.233333,16,0
7,5.696865,16.259921,0.083340,35,0
8,16.259921,23.513123,0.000011,12,0
9,23.513123,1.198840,0.000022,14,0


### NEW DATA WITH ALL NORMAL

In [25]:
key_list = []
sample_list = []
for i, j, k, l in zip(amplitudeBothEyesDictPrimaryNORMAL_new, amplitudeBothEyesDictSecondaryNORMAL_new, saccade_latencyNORMAL_new, correctiveSaccadeNORMAL_new):
    #looping in order 
    #extracting samples 
    #in dictionary 
    print(len(j.keys()))
    for key in j.keys():
        key_list.append(key)
        sample_list.append([i[key], j[key], k[key], l[key]])
        
df_normal_new = pd.DataFrame(sample_list)
df_normal_new['normal'] = 1
df_read = pd.read_csv('TestDataCompiled.csv')

np_array_original = df_read.values
np_array_new = df_normal_new.values

stacked_array = pd.DataFrame(np.vstack((np_array_original, np_array_new)))


print(df_normal_new)
print(stacked_array)

stacked_array.to_csv('NewTestDataCompiled.csv', index=False)


19
17
15
13
            0          1         2   3  normal
0   13.667422   2.641576  0.008333  15       1
1    1.430998   2.047580  0.100014  16       1
2   27.526416   9.393168  0.100010  20       1
3    7.648104  26.410982  0.108351  34       1
4   11.308977  19.349248  0.000022   1       1
5    3.538982   5.452786  0.058361  32       1
6    8.730061   0.290000  0.083373  35       1
7    7.821164   5.213651  0.000044  27       1
8    1.677145   1.677145  0.008325   4       1
9   17.419656   1.963730  0.308339  22       1
10   9.343647   1.295563  0.158346  17       1
11   0.677898   0.497337  0.025017  31       1
12   6.418217  12.283114  0.008357  15       1
13   9.391542   4.177183  0.000020  23       1
14  11.542588  12.434590  0.083368  23       1
15   8.620230   0.469188  0.066705  26       1
16   3.279299   3.856686  0.033379  24       1
17   6.718711   0.846894  0.000042  18       1
18   9.980406   1.312811  0.025049  28       1
19   4.217322   1.056792  0.008334   8       1
2

In [80]:
from scipy import stats

amplitudeBothEyesDictPrimaryLEFT = [value for dictionary in amplitudeBothEyesDictPrimaryLEFT for value in dictionary.values()]
amplitudeBothEyesDictSecondaryLEFT = [value for dictionary in amplitudeBothEyesDictSecondaryLEFT for value in dictionary.values()]
# amplitudeRightEyesDictPrimaryLEFT = [value for dictionary in amplitudeRightEyesDictPrimaryLEFT for value in dictionary.values()]
# amplitudeRightEyesDictSecondaryLEFT = [value for dictionary in amplitudeRightEyesDictSecondaryLEFT for value in dictionary.values()]
# amplitudeLeftEyesDictPrimaryLEFT = [value for dictionary in amplitudeLeftEyesDictPrimaryLEFT for value in dictionary.values()]
# amplitudeLeftEyesDictSecondaryLEFT = [value for dictionary in amplitudeLeftEyesDictSecondaryLEFT for value in dictionary.values()]
saccade_latencyLEFT = [value for dictionary in saccade_latencyLEFT for value in dictionary.values()]
# accuracy_distPrimaryLEFT = [value for dictionary in accuracy_distPrimaryLEFT for value in dictionary.values()]
# accuracy_distSecondaryLEFT = [value for dictionary in accuracy_distSecondaryLEFT for value in dictionary.values()]
correctiveSaccadeLEFT = [value for dictionary in correctiveSaccadeLEFT for value in dictionary.values()]

amplitudeBothEyesDictPrimaryRIGHT = [value for dictionary in amplitudeBothEyesDictPrimaryRIGHT for value in dictionary.values()]
amplitudeBothEyesDictSecondaryRIGHT= [value for dictionary in amplitudeBothEyesDictSecondaryRIGHT for value in dictionary.values()]
# amplitudeRightEyesDictPrimaryRIGHT = [value for dictionary in amplitudeRightEyesDictPrimaryRIGHT for value in dictionary.values()]
# amplitudeRightEyesDictSecondaryRIGHT= [value for dictionary in amplitudeRightEyesDictSecondaryRIGHT for value in dictionary.values()]
# amplitudeLeftEyesDictPrimaryRIGHT= [value for dictionary in amplitudeLeftEyesDictPrimaryRIGHT for value in dictionary.values()]
# amplitudeLeftEyesDictSecondaryRIGHT = [value for dictionary in amplitudeLeftEyesDictSecondaryRIGHT for value in dictionary.values()]
saccade_latencyRIGHT= [value for dictionary in saccade_latencyRIGHT for value in dictionary.values()]
# accuracy_distPrimaryRIGHT= [value for dictionary in accuracy_distPrimaryRIGHT for value in dictionary.values()]
# accuracy_distSecondaryRIGHT= [value for dictionary in accuracy_distSecondaryRIGHT for value in dictionary.values()]
correctiveSaccadeRIGHT= [value for dictionary in correctiveSaccadeRIGHT for value in dictionary.values()]


amplitudeBothEyesDictPrimaryNormal = [value for dictionary in amplitudeBothEyesDictPrimaryNormal for value in dictionary.values()]
amplitudeBothEyesDictSecondaryNormal= [value for dictionary in amplitudeBothEyesDictSecondaryNormal for value in dictionary.values()]
# amplitudeRightEyesDictPrimaryNormal = [value for dictionary in amplitudeRightEyesDictPrimaryNormal for value in dictionary.values()]
# amplitudeRightEyesDictSecondaryNormal= [value for dictionary in amplitudeRightEyesDictSecondaryNormal for value in dictionary.values()]
# amplitudeLeftEyesDictPrimaryNormal= [value for dictionary in amplitudeLeftEyesDictPrimaryNormal for value in dictionary.values()]
# amplitudeLeftEyesDictSecondaryNormal = [value for dictionary in amplitudeLeftEyesDictSecondaryNormal for value in dictionary.values()]
saccade_latencyNormal = [value for dictionary in saccade_latencyNormal for value in dictionary.values()]
# accuracy_distPrimaryNormal = [value for dictionary in accuracy_distPrimaryNormal for value in dictionary.values()]
# accuracy_distSecondaryNormal= [value for dictionary in accuracy_distSecondaryNormal for value in dictionary.values()]
correctiveSaccadeNormal= [value for dictionary in correctiveSaccadeNormal for value in dictionary.values()]



In [40]:
df_abnormal = pd.DataFrame()
print(len(amplitudeBothEyesDictPrimaryRIGHT))
print(len(amplitudeBothEyesDictPrimaryLEFT))
print(len(amplitudeBothEyesDictSecondaryRIGHT))
print(len(amplitudeBothEyesDictSecondaryLEFT))
print(len(saccade_latencyRIGHT))
print(len(saccade_latencyLEFT))
print(len(correctiveSaccadeRIGHT))
print(len(correctiveSaccadeLEFT))

print(len(amplitudeBothEyesDictPrimaryNormal))
print(len(amplitudeBothEyesDictSecondaryNormal))
print(len(saccade_latencyNormal))
print(len(correctiveSaccadeNormal))


df_abnormal['amplitudePrimary'] = amplitudeBothEyesDictPrimaryRIGHT.extend(amplitudeBothEyesDictPrimaryLEFT)
df_abnormal['amplitudeSecondary'] = amplitudeBothEyesDictSecondaryRIGHT.extend(amplitudeBothEyesDictSecondaryLEFT)
df_abnormal['saccadeLatency'] = saccade_latencyRIGHT.extend(saccade_latencyLEFT)
df_abnormal['correctiveSaccade'] = correctiveSaccadeRIGHT.extend(correctiveSaccadeLEFT)

df_normal = pd.DataFrame()
df_normal['amplitudePrimary'] = amplitudeBothEyesDictPrimaryNormal
df_normal['amplitudeSecondary'] = amplitudeBothEyesDictSecondaryNormal
df_normal['saccadeLatency'] = saccade_latencyNormal
df_normal['correctiveSaccade'] = correctiveSaccadeNormal

display(df_normal)
display(df_abnormal)




880
110
539
67
880
110
539
67
109
55
110
55


ValueError: Length of values (55) does not match length of index (109)

In [ ]:
# Fields for the dataframe: amplitudeBothEyesDictPrimaryRIGHT, amplitudeBothEyesDictSecondaryRIGHT, saccade_latencyRIGHT, correctiveSaccadeRIGHT

#Making the df out of these features - with the proper target variables

# saccade_latencyRIGHT.append(dataCollection['saccade_latency'])
# correctiveSaccadeRIGHT.append(dataCollection['correctiveSaccade'])
# amplitudeBothEyesDictPrimaryLEFT

### Stats

In [40]:
print(amplitudeBothEyesDictPrimaryLEFT)

[7.905334633942032, 17.381434778105046, 29.4155353553435, 19.747376896335798, 10.173487499918533, 6.801426237643962, 7.7333782761743874, 7.519674444095064, 13.851052659517958, 14.642191956445604, 15.496671173867338, 12.924695533198388, 5.6968654961516, 16.259921268855024, 23.513122995108052, 21.909296183266633, 15.020333630327656, 21.560429178894605, 16.65955454989199, 5.3977792711777575, 16.849943032547642, 17.107161171003323, 0.4003521919650133, 18.09838019884712, 16.640672639249747, 0.6009277768624848, 23.327565553813553, 1.4873979828298396, 18.854921382282686, 17.687999593206424, 31.80627830391118, 16.85088307657873, 0.6898319803870897, 1.155793244433404, 18.09838019884712, 15.694188089694785, 30.903176662968207, 19.113456863013553, 19.726215179267097, 32.813336688397584, 17.139463787513474, 20.347048844812, 31.416255835665197, 17.260865415413985, 1.7655573415719328, 16.83325950656756, 30.27090314731632, 32.72295241551905, 30.99049848590557, 20.953670841321237, 17.18433608162732, 1

In [71]:
# print(amplitudeBothEyesDictPrimaryLEFT,amplitudeBothEyesDictPrimaryNormal, amplitudeBothEyesDictPrimaryRIGHT)
stat, p_value = stats.mannwhitneyu(amplitudeBothEyesDictPrimaryLEFT, amplitudeBothEyesDictPrimaryRIGHT)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(amplitudeBothEyesDictPrimaryNormal, amplitudeBothEyesDictPrimaryRIGHT)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(amplitudeBothEyesDictPrimaryLEFT, amplitudeBothEyesDictPrimaryNormal)
print(stat, p_value)

print(" ")

# amplitudeRightEyesDictSecondaryLEFT = [value for value in amplitudeRightEyesDictSecondaryLEFT if not np.isnan(value)]
# amplitudeRightEyesDictSecondaryRIGHT = [value for value in amplitudeRightEyesDictSecondaryRIGHT if not np.isnan(value)]
# amplitudeRightEyesDictSecondaryNormal = [value for value in amplitudeRightEyesDictSecondaryNormal if not np.isnan(value)]


# amplitudeLeftEyesDictPrimaryLEFT = [value for value in amplitudeLeftEyesDictPrimaryLEFT if not np.isnan(value)]
# amplitudeLeftEyesDictPrimaryRIGHT = [value for value in amplitudeLeftEyesDictPrimaryRIGHT if not np.isnan(value)]
# amplitudeLeftEyesDictPrimaryNormal = [value for value in amplitudeLeftEyesDictPrimaryNormal if not np.isnan(value)]

# amplitudeRightEyesDictPrimaryLEFT = [value for value in amplitudeRightEyesDictPrimaryLEFT if not np.isnan(value)]
# amplitudeRightEyesDictPrimaryRIGHT = [value for value in amplitudeRightEyesDictPrimaryRIGHT if not np.isnan(value)]
# amplitudeRightEyesDictPrimaryNormal = [value for value in amplitudeRightEyesDictPrimaryNormal if not np.isnan(value)]



# stat, p_value = stats.mannwhitneyu(amplitudeRightEyesDictPrimaryLEFT, amplitudeRightEyesDictPrimaryRIGHT)
# print(stat, p_value)
# stat, p_value = stats.mannwhitneyu(amplitudeRightEyesDictPrimaryNormal, amplitudeRightEyesDictPrimaryRIGHT)
# print(stat, p_value)
# stat, p_value = stats.mannwhitneyu(amplitudeRightEyesDictPrimaryLEFT, amplitudeRightEyesDictPrimaryNormal)
# print(stat, p_value)

# print(" ")
# stat, p_value = stats.mannwhitneyu(amplitudeLeftEyesDictPrimaryLEFT, amplitudeLeftEyesDictPrimaryRIGHT)
# print(stat, p_value)
# stat, p_value = stats.mannwhitneyu(amplitudeLeftEyesDictPrimaryNormal, amplitudeLeftEyesDictPrimaryRIGHT)
# print(stat, p_value)
# stat, p_value = stats.mannwhitneyu(amplitudeLeftEyesDictPrimaryLEFT, amplitudeLeftEyesDictPrimaryNormal)
# print(stat, p_value)
# print(" ")

# amplitudeRightEyesDictSecondaryLEFT = [value for value in amplitudeRightEyesDictSecondaryLEFT if not np.isnan(value)]
# amplitudeRightEyesDictSecondaryRIGHT = [value for value in amplitudeRightEyesDictSecondaryRIGHT if not np.isnan(value)]
# amplitudeRightEyesDictSecondaryNormal = [value for value in amplitudeRightEyesDictSecondaryNormal if not np.isnan(value)]

# amplitudeLeftEyesDictSecondaryLEFT = [value for value in amplitudeLeftEyesDictSecondaryLEFT if not np.isnan(value)]
# amplitudeLeftEyesDictSecondaryRIGHT = [value for value in amplitudeLeftEyesDictSecondaryRIGHT if not np.isnan(value)]
# amplitudeLeftEyesDictSecondaryNormal = [value for value in amplitudeLeftEyesDictSecondaryNormal if not np.isnan(value)]


# stat, p_value = stats.mannwhitneyu(amplitudeLeftEyesDictSecondaryLEFT, amplitudeLeftEyesDictSecondaryRIGHT)
# print(stat, p_value)
# stat, p_value = stats.mannwhitneyu(amplitudeLeftEyesDictSecondaryNormal, amplitudeLeftEyesDictSecondaryRIGHT)
# print(stat, p_value)
# stat, p_value = stats.mannwhitneyu(amplitudeLeftEyesDictSecondaryLEFT, amplitudeLeftEyesDictSecondaryNormal)
# print(stat, p_value)
# print(" ")
# stat, p_value = stats.mannwhitneyu(amplitudeRightEyesDictSecondaryLEFT, amplitudeRightEyesDictSecondaryNormal)
# print(stat, p_value)
# stat, p_value = stats.mannwhitneyu(amplitudeRightEyesDictSecondaryNormal, amplitudeRightEyesDictSecondaryRIGHT)
# print(stat, p_value)
# stat, p_value = stats.mannwhitneyu(amplitudeRightEyesDictSecondaryLEFT, amplitudeRightEyesDictSecondaryNormal)
# print(stat, p_value)
# print(" ")

stat, p_value = stats.mannwhitneyu(amplitudeBothEyesDictSecondaryLEFT, amplitudeBothEyesDictSecondaryRIGHT)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(amplitudeBothEyesDictSecondaryNormal, amplitudeBothEyesDictSecondaryRIGHT)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(amplitudeBothEyesDictSecondaryLEFT, amplitudeBothEyesDictSecondaryNormal)
print(stat, p_value)
print(" ")
stat, p_value = stats.mannwhitneyu(saccade_latencyLEFT, saccade_latencyRIGHT)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(saccade_latencyNormal, saccade_latencyRIGHT)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(saccade_latencyLEFT, saccade_latencyNormal)
print(stat, p_value)
print(" ")
# stat, p_value = stats.mannwhitneyu(accuracy_distPrimaryLEFT, accuracy_distPrimaryRIGHT)
# print(stat, p_value)
# stat, p_value = stats.mannwhitneyu(accuracy_distPrimaryNormal, accuracy_distPrimaryRIGHT)
# print(stat, p_value)
# stat, p_value = stats.mannwhitneyu(accuracy_distPrimaryLEFT, accuracy_distPrimaryNormal)
# print(stat, p_value)
# print(" ")
# stat, p_value = stats.mannwhitneyu(accuracy_distSecondaryLEFT, accuracy_distSecondaryRIGHT)
# print(stat, p_value)
# stat, p_value = stats.mannwhitneyu(accuracy_distSecondaryNormal, accuracy_distSecondaryRIGHT)
# print(stat, p_value)
# stat, p_value = stats.mannwhitneyu(accuracy_distSecondaryLEFT, accuracy_distSecondaryNormal)
# print(stat, p_value)

print(" ")
stat, p_value = stats.mannwhitneyu(correctiveSaccadeLEFT, correctiveSaccadeRIGHT)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(correctiveSaccadeNormal, correctiveSaccadeRIGHT)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(correctiveSaccadeLEFT, correctiveSaccadeNormal)
# print(correctiveSaccadeNormal)
# print(correctiveSaccadeLEFT)
# print(correctiveSaccadeRIGHT)
print(stat, p_value)

5952.0 0.8363682437629498
8828.0 0.2396265832292811
10482.0 0.20604975444996043
 
2527.5 0.4332297050380418
4044.5 0.31268710435281855
3442.0 0.7352813848132482
 
6658.5 0.19775539577041523
8891.5 0.24684742207383892
11442.5 0.00961356017295379
 
 
1959.5 0.09699860559494365
3138.5 0.08402224151108766
3512.5 0.9056251225279153


In [81]:
amplitudeBothEyesDictPrimaryLEFT.extend(amplitudeBothEyesDictPrimaryRIGHT)
amplitudeBothEyesDictSecondaryLEFT.extend(amplitudeBothEyesDictSecondaryRIGHT)
saccade_latencyLEFT.extend(saccade_latencyRIGHT)
correctiveSaccadeLEFT.extend(correctiveSaccadeRIGHT)


In [82]:
print(amplitudeBothEyesDictPrimaryLEFT,amplitudeBothEyesDictPrimaryNormal, amplitudeBothEyesDictPrimaryRIGHT)


# stat, p_value = stats.mannwhitneyu(amplitudeBothEyesDictPrimaryLEFT, amplitudeBothEyesDictPrimaryRIGHT)
# print(stat, p_value)
# stat, p_value = stats.mannwhitneyu(amplitudeBothEyesDictPrimaryNormal, amplitudeBothEyesDictPrimaryRIGHT)
# print(stat, p_value)
stat, p_value = stats.mannwhitneyu(amplitudeBothEyesDictPrimaryLEFT, amplitudeBothEyesDictPrimaryNormal)
print(stat, p_value)

print(" ")

# amplitudeRightEyesDictSecondaryLEFT = [value for value in amplitudeRightEyesDictSecondaryLEFT if not np.isnan(value)]
# amplitudeRightEyesDictSecondaryRIGHT = [value for value in amplitudeRightEyesDictSecondaryRIGHT if not np.isnan(value)]
# amplitudeRightEyesDictSecondaryNormal = [value for value in amplitudeRightEyesDictSecondaryNormal if not np.isnan(value)]


# amplitudeLeftEyesDictPrimaryLEFT = [value for value in amplitudeLeftEyesDictPrimaryLEFT if not np.isnan(value)]
# amplitudeLeftEyesDictPrimaryRIGHT = [value for value in amplitudeLeftEyesDictPrimaryRIGHT if not np.isnan(value)]
# amplitudeLeftEyesDictPrimaryNormal = [value for value in amplitudeLeftEyesDictPrimaryNormal if not np.isnan(value)]

# amplitudeRightEyesDictPrimaryLEFT = [value for value in amplitudeRightEyesDictPrimaryLEFT if not np.isnan(value)]
# amplitudeRightEyesDictPrimaryRIGHT = [value for value in amplitudeRightEyesDictPrimaryRIGHT if not np.isnan(value)]
# amplitudeRightEyesDictPrimaryNormal = [value for value in amplitudeRightEyesDictPrimaryNormal if not np.isnan(value)]



# stat, p_value = stats.mannwhitneyu(amplitudeRightEyesDictPrimaryLEFT, amplitudeRightEyesDictPrimaryRIGHT)
# print(stat, p_value)
# stat, p_value = stats.mannwhitneyu(amplitudeRightEyesDictPrimaryNormal, amplitudeRightEyesDictPrimaryRIGHT)
# print(stat, p_value)
# stat, p_value = stats.mannwhitneyu(amplitudeRightEyesDictPrimaryLEFT, amplitudeRightEyesDictPrimaryNormal)
# print(stat, p_value)

# print(" ")
# stat, p_value = stats.mannwhitneyu(amplitudeLeftEyesDictPrimaryLEFT, amplitudeLeftEyesDictPrimaryRIGHT)
# print(stat, p_value)
# stat, p_value = stats.mannwhitneyu(amplitudeLeftEyesDictPrimaryNormal, amplitudeLeftEyesDictPrimaryRIGHT)
# print(stat, p_value)
# stat, p_value = stats.mannwhitneyu(amplitudeLeftEyesDictPrimaryLEFT, amplitudeLeftEyesDictPrimaryNormal)
# print(stat, p_value)
# print(" ")

# amplitudeRightEyesDictSecondaryLEFT = [value for value in amplitudeRightEyesDictSecondaryLEFT if not np.isnan(value)]
# amplitudeRightEyesDictSecondaryRIGHT = [value for value in amplitudeRightEyesDictSecondaryRIGHT if not np.isnan(value)]
# amplitudeRightEyesDictSecondaryNormal = [value for value in amplitudeRightEyesDictSecondaryNormal if not np.isnan(value)]

# amplitudeLeftEyesDictSecondaryLEFT = [value for value in amplitudeLeftEyesDictSecondaryLEFT if not np.isnan(value)]
# amplitudeLeftEyesDictSecondaryRIGHT = [value for value in amplitudeLeftEyesDictSecondaryRIGHT if not np.isnan(value)]
# amplitudeLeftEyesDictSecondaryNormal = [value for value in amplitudeLeftEyesDictSecondaryNormal if not np.isnan(value)]


# stat, p_value = stats.mannwhitneyu(amplitudeLeftEyesDictSecondaryLEFT, amplitudeLeftEyesDictSecondaryRIGHT)
# print(stat, p_value)
# stat, p_value = stats.mannwhitneyu(amplitudeLeftEyesDictSecondaryNormal, amplitudeLeftEyesDictSecondaryRIGHT)
# print(stat, p_value)
# stat, p_value = stats.mannwhitneyu(amplitudeLeftEyesDictSecondaryLEFT, amplitudeLeftEyesDictSecondaryNormal)
# print(stat, p_value)
# print(" ")
# stat, p_value = stats.mannwhitneyu(amplitudeRightEyesDictSecondaryLEFT, amplitudeRightEyesDictSecondaryNormal)
# print(stat, p_value)
# stat, p_value = stats.mannwhitneyu(amplitudeRightEyesDictSecondaryNormal, amplitudeRightEyesDictSecondaryRIGHT)
# print(stat, p_value)
# stat, p_value = stats.mannwhitneyu(amplitudeRightEyesDictSecondaryLEFT, amplitudeRightEyesDictSecondaryNormal)
# print(stat, p_value)
# print(" ")

# stat, p_value = stats.mannwhitneyu(amplitudeBothEyesDictSecondaryLEFT, amplitudeBothEyesDictSecondaryRIGHT)
# print(stat, p_value)
# stat, p_value = stats.mannwhitneyu(amplitudeBothEyesDictSecondaryNormal, amplitudeBothEyesDictSecondaryRIGHT)
# print(stat, p_value)
stat, p_value = stats.mannwhitneyu(amplitudeBothEyesDictSecondaryLEFT, amplitudeBothEyesDictSecondaryNormal)
print(stat, p_value)
print(" ")
# stat, p_value = stats.mannwhitneyu(saccade_latencyLEFT, saccade_latencyRIGHT)
# print(stat, p_value)
# stat, p_value = stats.mannwhitneyu(saccade_latencyNormal, saccade_latencyRIGHT)
# print(stat, p_value)
stat, p_value = stats.mannwhitneyu(saccade_latencyLEFT, saccade_latencyNormal)
print(stat, p_value)
print(" ")
# stat, p_value = stats.mannwhitneyu(accuracy_distPrimaryLEFT, accuracy_distPrimaryRIGHT)
# print(stat, p_value)
# stat, p_value = stats.mannwhitneyu(accuracy_distPrimaryNormal, accuracy_distPrimaryRIGHT)
# print(stat, p_value)
# stat, p_value = stats.mannwhitneyu(accuracy_distPrimaryLEFT, accuracy_distPrimaryNormal)
# print(stat, p_value)
# print(" ")
# stat, p_value = stats.mannwhitneyu(accuracy_distSecondaryLEFT, accuracy_distSecondaryRIGHT)
# print(stat, p_value)
# stat, p_value = stats.mannwhitneyu(accuracy_distSecondaryNormal, accuracy_distSecondaryRIGHT)
# print(stat, p_value)
# stat, p_value = stats.mannwhitneyu(accuracy_distSecondaryLEFT, accuracy_distSecondaryNormal)
# print(stat, p_value)

print(" ")
# stat, p_value = stats.mannwhitneyu(correctiveSaccadeLEFT, correctiveSaccadeRIGHT)
# print(stat, p_value)
# stat, p_value = stats.mannwhitneyu(correctiveSaccadeNormal, correctiveSaccadeRIGHT)
# print(stat, p_value)
stat, p_value = stats.mannwhitneyu(correctiveSaccadeLEFT, correctiveSaccadeNormal)
# print(correctiveSaccadeNormal)
# print(correctiveSaccadeLEFT)
# print(correctiveSaccadeRIGHT)
print(stat, p_value)

[7.905334633942032, 17.381434778105046, 29.4155353553435, 19.747376896335798, 10.173487499918533, 6.801426237643962, 7.7333782761743874, 7.519674444095064, 13.851052659517958, 14.642191956445604, 15.496671173867338, 12.924695533198388, 5.6968654961516, 16.259921268855024, 23.513122995108052, 21.909296183266633, 15.020333630327656, 21.560429178894605, 16.65955454989199, 5.3977792711777575, 16.849943032547642, 17.107161171003323, 0.4003521919650133, 18.09838019884712, 16.640672639249747, 0.6009277768624848, 23.327565553813553, 1.4873979828298396, 18.854921382282686, 17.687999593206424, 31.80627830391118, 16.85088307657873, 0.6898319803870897, 1.155793244433404, 18.09838019884712, 15.694188089694785, 30.903176662968207, 19.113456863013553, 19.726215179267097, 32.813336688397584, 17.139463787513474, 20.347048844812, 31.416255835665197, 17.260865415413985, 1.7655573415719328, 16.83325950656756, 30.27090314731632, 32.72295241551905, 30.99049848590557, 20.953670841321237, 17.18433608162732, 1

In [42]:
print(" ")
stat, p_value = stats.mannwhitneyu(amplitudeLeftEyesDictPrimaryLEFT, amplitudeRightEyesDictPrimaryLEFT)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(amplitudeLeftEyesDictPrimaryLEFT, amplitudeBothEyesDictPrimaryLEFT)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(amplitudeRightEyesDictPrimaryLEFT, amplitudeBothEyesDictPrimaryLEFT)
print(stat, p_value)
print(" ")
print(" ")

stat, p_value = stats.mannwhitneyu(amplitudeLeftEyesDictPrimaryRIGHT, amplitudeRightEyesDictPrimaryRIGHT)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(amplitudeLeftEyesDictPrimaryRIGHT, amplitudeBothEyesDictPrimaryRIGHT)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(amplitudeRightEyesDictPrimaryRIGHT, amplitudeBothEyesDictPrimaryRIGHT)
print(stat, p_value)
print(" ")

stat, p_value = stats.mannwhitneyu(amplitudeLeftEyesDictPrimaryNormal, amplitudeRightEyesDictPrimaryNormal)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(amplitudeLeftEyesDictPrimaryNormal, amplitudeBothEyesDictPrimaryNormal)
print(stat, p_value)
stat, p_value = stats.mannwhitneyu(amplitudeRightEyesDictPrimaryNormal, amplitudeBothEyesDictPrimaryNormal)
print(stat, p_value)
print(" ")

 
4316.0 0.43348457461110446
4305.0 0.4170037066927256
6056.0 0.9907039144845645
 
 
6229.0 0.5355260957266295
6206.0 0.741846895232644
5777.0 0.7270963779504298
 
5730.5 0.7374945598189819
5935.5 0.9922894029326551
6038.0 0.7432107055217011
 


In [43]:
from scipy.stats import kruskal
# print(amplitudeBothEyesDictPrimaryLEFT,amplitudeBothEyesDictPrimaryNormal, amplitudeBothEyesDictPrimaryRIGHT)
stat, p_value = kruskal(amplitudeBothEyesDictPrimaryLEFT, amplitudeBothEyesDictPrimaryRIGHT, amplitudeBothEyesDictPrimaryNormal)
print(stat, p_value)
# stat, p_value = stats.mannwhitneyu(amplitudeBothEyesDictPrimaryNormal, amplitudeBothEyesDictPrimaryRIGHT)
# print(stat, p_value)
# stat, p_value = stats.mannwhitneyu(amplitudeBothEyesDictPrimaryLEFT, amplitudeBothEyesDictPrimaryNormal)
# print(stat, p_value)

print(" ")

amplitudeRightEyesDictSecondaryLEFT = [value for value in amplitudeRightEyesDictSecondaryLEFT if not np.isnan(value)]
amplitudeRightEyesDictSecondaryRIGHT = [value for value in amplitudeRightEyesDictSecondaryRIGHT if not np.isnan(value)]
amplitudeRightEyesDictSecondaryNormal = [value for value in amplitudeRightEyesDictSecondaryNormal if not np.isnan(value)]


amplitudeLeftEyesDictPrimaryLEFT = [value for value in amplitudeLeftEyesDictPrimaryLEFT if not np.isnan(value)]
amplitudeLeftEyesDictPrimaryRIGHT = [value for value in amplitudeLeftEyesDictPrimaryRIGHT if not np.isnan(value)]
amplitudeLeftEyesDictPrimaryNormal = [value for value in amplitudeLeftEyesDictPrimaryNormal if not np.isnan(value)]

amplitudeRightEyesDictPrimaryLEFT = [value for value in amplitudeRightEyesDictPrimaryLEFT if not np.isnan(value)]
amplitudeRightEyesDictPrimaryRIGHT = [value for value in amplitudeRightEyesDictPrimaryRIGHT if not np.isnan(value)]
amplitudeRightEyesDictPrimaryNormal = [value for value in amplitudeRightEyesDictPrimaryNormal if not np.isnan(value)]



stat, p_value = kruskal(amplitudeRightEyesDictPrimaryLEFT, amplitudeRightEyesDictPrimaryRIGHT, amplitudeRightEyesDictPrimaryNormal)
print(stat, p_value)
# stat, p_value = stats.mannwhitneyu(amplitudeRightEyesDictPrimaryNormal, amplitudeRightEyesDictPrimaryRIGHT)
# print(stat, p_value)
# stat, p_value = stats.mannwhitneyu(amplitudeRightEyesDictPrimaryLEFT, amplitudeRightEyesDictPrimaryNormal)
# print(stat, p_value)

print(" ")
stat, p_value = kruskal(amplitudeLeftEyesDictPrimaryLEFT, amplitudeLeftEyesDictPrimaryRIGHT, amplitudeLeftEyesDictPrimaryNormal)
# print(stat, p_value)
# stat, p_value = stats.mannwhitneyu(amplitudeLeftEyesDictPrimaryNormal, amplitudeLeftEyesDictPrimaryRIGHT)
# print(stat, p_value)
# stat, p_value = stats.mannwhitneyu(amplitudeLeftEyesDictPrimaryLEFT, amplitudeLeftEyesDictPrimaryNormal)
# print(stat, p_value)
print(" ")

amplitudeRightEyesDictSecondaryLEFT = [value for value in amplitudeRightEyesDictSecondaryLEFT if not np.isnan(value)]
amplitudeRightEyesDictSecondaryRIGHT = [value for value in amplitudeRightEyesDictSecondaryRIGHT if not np.isnan(value)]
amplitudeRightEyesDictSecondaryNormal = [value for value in amplitudeRightEyesDictSecondaryNormal if not np.isnan(value)]

amplitudeLeftEyesDictSecondaryLEFT = [value for value in amplitudeLeftEyesDictSecondaryLEFT if not np.isnan(value)]
amplitudeLeftEyesDictSecondaryRIGHT = [value for value in amplitudeLeftEyesDictSecondaryRIGHT if not np.isnan(value)]
amplitudeLeftEyesDictSecondaryNormal = [value for value in amplitudeLeftEyesDictSecondaryNormal if not np.isnan(value)]


stat, p_value = kruskal(amplitudeLeftEyesDictSecondaryLEFT, amplitudeLeftEyesDictSecondaryRIGHT, amplitudeLeftEyesDictSecondaryNormal)
print(stat, p_value)
# stat, p_value = stats.mannwhitneyu(amplitudeLeftEyesDictSecondaryNormal, amplitudeLeftEyesDictSecondaryRIGHT)
# print(stat, p_value)
# stat, p_value = stats.mannwhitneyu(amplitudeLeftEyesDictSecondaryLEFT, amplitudeLeftEyesDictSecondaryNormal)
# print(stat, p_value)
print(" ")

stat, p_value = kruskal(amplitudeRightEyesDictSecondaryLEFT, amplitudeRightEyesDictSecondaryRIGHT, amplitudeRightEyesDictSecondaryNormal)
print(stat, p_value)


stat, p_value = kruskal(amplitudeBothEyesDictSecondaryLEFT, amplitudeBothEyesDictSecondaryRIGHT, amplitudeBothEyesDictSecondaryNormal)
print(stat, p_value)
print(" ")


stat, p_value = kruskal(saccade_latencyLEFT, saccade_latencyRIGHT, saccade_latencyNormal)
print(stat, p_value)
print(" ")


stat, p_value = kruskal(accuracy_distPrimaryLEFT, accuracy_distPrimaryRIGHT, accuracy_distPrimaryNormal)
print(stat, p_value)
print(" ")

stat, p_value = kruskal(accuracy_distSecondaryLEFT, accuracy_distSecondaryRIGHT, accuracy_distSecondaryNormal)
print(stat, p_value)
print(" ")

stat, p_value = kruskal(correctiveSaccadeLEFT, correctiveSaccadeRIGHT, correctiveSaccadeNormal)
print(stat, p_value)
print(" ")



0.2447423739480273 0.8848198739283947
 
1.0815862208137899 0.5822862519033402
 
 
1.1641699508178651 0.558732208367647
 
2.110325426827825 0.34813577633671344
0.5785525543837647 0.7488052989969515
 
1.7298079351455131 0.4210919891714934
 
0.3360880037038214 0.8453166383687722
 
0.9540031342681914 0.6206415566641306
 
10.930428196332416 0.004231435007569993
 


In [44]:
from scipy.stats import wilcoxon

print(amplitudeBothEyesDictPrimaryLEFT,amplitudeBothEyesDictPrimaryNormal, amplitudeBothEyesDictPrimaryRIGHT)
stat, p_value = wilcoxon(amplitudeBothEyesDictPrimaryLEFT, amplitudeBothEyesDictPrimaryRIGHT)
print(stat, p_value)
stat, p_value = wilcoxon(amplitudeBothEyesDictPrimaryNormal, amplitudeBothEyesDictPrimaryRIGHT)
print(stat, p_value)
stat, p_value = wilcoxon(amplitudeBothEyesDictPrimaryLEFT, amplitudeBothEyesDictPrimaryNormal)
print(stat, p_value)

print(" ")

amplitudeRightEyesDictSecondaryLEFT = [value for value in amplitudeRightEyesDictSecondaryLEFT if not np.isnan(value)]
amplitudeRightEyesDictSecondaryRIGHT = [value for value in amplitudeRightEyesDictSecondaryRIGHT if not np.isnan(value)]
amplitudeRightEyesDictSecondaryNormal = [value for value in amplitudeRightEyesDictSecondaryNormal if not np.isnan(value)]


amplitudeLeftEyesDictPrimaryLEFT = [value for value in amplitudeLeftEyesDictPrimaryLEFT if not np.isnan(value)]
amplitudeLeftEyesDictPrimaryRIGHT = [value for value in amplitudeLeftEyesDictPrimaryRIGHT if not np.isnan(value)]
amplitudeLeftEyesDictPrimaryNormal = [value for value in amplitudeLeftEyesDictPrimaryNormal if not np.isnan(value)]

amplitudeRightEyesDictPrimaryLEFT = [value for value in amplitudeRightEyesDictPrimaryLEFT if not np.isnan(value)]
amplitudeRightEyesDictPrimaryRIGHT = [value for value in amplitudeRightEyesDictPrimaryRIGHT if not np.isnan(value)]
amplitudeRightEyesDictPrimaryNormal = [value for value in amplitudeRightEyesDictPrimaryNormal if not np.isnan(value)]



stat, p_value = wilcoxon(amplitudeRightEyesDictPrimaryLEFT, amplitudeRightEyesDictPrimaryRIGHT)
print(stat, p_value)
stat, p_value = wilcoxon(amplitudeRightEyesDictPrimaryNormal, amplitudeRightEyesDictPrimaryRIGHT)
print(stat, p_value)
stat, p_value = wilcoxon(amplitudeRightEyesDictPrimaryLEFT, amplitudeRightEyesDictPrimaryNormal)
print(stat, p_value)

print(" ")
stat, p_value = wilcoxon(amplitudeLeftEyesDictPrimaryLEFT, amplitudeLeftEyesDictPrimaryRIGHT)
print(stat, p_value)
stat, p_value = wilcoxon(amplitudeLeftEyesDictPrimaryNormal, amplitudeLeftEyesDictPrimaryRIGHT)
print(stat, p_value)
stat, p_value = wilcoxon(amplitudeLeftEyesDictPrimaryLEFT, amplitudeLeftEyesDictPrimaryNormal)
print(stat, p_value)
print(" ")

amplitudeRightEyesDictSecondaryLEFT = [value for value in amplitudeRightEyesDictSecondaryLEFT if not np.isnan(value)]
amplitudeRightEyesDictSecondaryRIGHT = [value for value in amplitudeRightEyesDictSecondaryRIGHT if not np.isnan(value)]
amplitudeRightEyesDictSecondaryNormal = [value for value in amplitudeRightEyesDictSecondaryNormal if not np.isnan(value)]

amplitudeLeftEyesDictSecondaryLEFT = [value for value in amplitudeLeftEyesDictSecondaryLEFT if not np.isnan(value)]
amplitudeLeftEyesDictSecondaryRIGHT = [value for value in amplitudeLeftEyesDictSecondaryRIGHT if not np.isnan(value)]
amplitudeLeftEyesDictSecondaryNormal = [value for value in amplitudeLeftEyesDictSecondaryNormal if not np.isnan(value)]


stat, p_value = wilcoxon(amplitudeLeftEyesDictSecondaryLEFT, amplitudeLeftEyesDictSecondaryRIGHT)
print(stat, p_value)
stat, p_value = wilcoxon(amplitudeLeftEyesDictSecondaryNormal, amplitudeLeftEyesDictSecondaryRIGHT)
print(stat, p_value)
stat, p_value = wilcoxon(amplitudeLeftEyesDictSecondaryLEFT, amplitudeLeftEyesDictSecondaryNormal)
print(stat, p_value)
print(" ")
stat, p_value =wilcoxon(amplitudeRightEyesDictSecondaryLEFT, amplitudeRightEyesDictSecondaryNormal)
print(stat, p_value)
stat, p_value = wilcoxon(amplitudeRightEyesDictSecondaryNormal, amplitudeRightEyesDictSecondaryRIGHT)
print(stat, p_value)
stat, p_value = wilcoxon(amplitudeRightEyesDictSecondaryLEFT, amplitudeRightEyesDictSecondaryNormal)
print(stat, p_value)
print(" ")

stat, p_value = wilcoxon(amplitudeBothEyesDictSecondaryLEFT, amplitudeBothEyesDictSecondaryRIGHT)
print(stat, p_value)
stat, p_value = wilcoxon(amplitudeBothEyesDictSecondaryNormal, amplitudeBothEyesDictSecondaryRIGHT)
print(stat, p_value)
stat, p_value = wilcoxon(amplitudeBothEyesDictSecondaryLEFT, amplitudeBothEyesDictSecondaryNormal)
print(stat, p_value)
print(" ")
stat, p_value = wilcoxon(saccade_latencyLEFT, saccade_latencyRIGHT)
print(stat, p_value)
stat, p_value = wilcoxon(saccade_latencyNormal, saccade_latencyRIGHT)
print(stat, p_value)
stat, p_value = wilcoxon(saccade_latencyLEFT, saccade_latencyNormal)
print(stat, p_value)
print(" ")
stat, p_value = wilcoxon(accuracy_distPrimaryLEFT, accuracy_distPrimaryRIGHT)
print(stat, p_value)
stat, p_value = wilcoxon(accuracy_distPrimaryNormal, accuracy_distPrimaryRIGHT)
print(stat, p_value)
stat, p_value = wilcoxon(accuracy_distPrimaryLEFT, accuracy_distPrimaryNormal)
print(stat, p_value)
print(" ")
stat, p_value = wilcoxon(accuracy_distSecondaryLEFT, accuracy_distSecondaryRIGHT)
print(stat, p_value)
stat, p_value = wilcoxon(accuracy_distSecondaryNormal, accuracy_distSecondaryRIGHT)
print(stat, p_value)
stat, p_value = wilcoxon(accuracy_distSecondaryLEFT, accuracy_distSecondaryNormal)
print(stat, p_value)

print(" ")
stat, p_value = wilcoxon(correctiveSaccadeLEFT, correctiveSaccadeRIGHT)
print(stat, p_value)
stat, p_value = wilcoxon(correctiveSaccadeNormal, correctiveSaccadeRIGHT)
print(stat, p_value)
stat, p_value = wilcoxon(correctiveSaccadeLEFT, correctiveSaccadeNormal)
print(correctiveSaccadeNormal)
print(correctiveSaccadeLEFT)
print(stat, p_value)

[7.905334633942032, 17.381434778105046, 29.4155353553435, 19.747376896335798, 10.173487499918533, 6.801426237643962, 7.7333782761743874, 7.519674444095064, 13.851052659517958, 14.642191956445604, 15.496671173867338, 12.924695533198388, 5.6968654961516, 16.259921268855024, 23.513122995108052, 21.909296183266633, 15.020333630327656, 21.560429178894605, 16.65955454989199, 5.3977792711777575, 16.849943032547642, 17.107161171003323, 0.4003521919650133, 18.09838019884712, 16.640672639249747, 0.6009277768624848, 23.327565553813553, 1.4873979828298396, 18.854921382282686, 17.687999593206424, 31.80627830391118, 16.85088307657873, 0.6898319803870897, 1.155793244433404, 18.09838019884712, 15.694188089694785, 30.903176662968207, 19.113456863013553, 19.726215179267097, 32.813336688397584, 17.139463787513474, 20.347048844812, 31.416255835665197, 17.260865415413985, 1.7655573415719328, 16.83325950656756, 30.27090314731632, 32.72295241551905, 30.99049848590557, 20.953670841321237, 17.18433608162732, 1

ValueError: The samples x and y must have the same length.

In [ ]:
# for key in plusLenseRight175.keys():  # Assuming keys are the same for all dictionaries
#     for i in range(len(dataCollection)):
#         for j in range(i + 1, len(dataCollection)):
#             group1 = list(dataCollection[i][key].values())
        
#             group2 = list(dataCollection[j][key].values())
#             # Perform the Kolmogorov-Smirnov test
# #             stat, p_value = stats.ks_2samp(group1, group2)
# #             print(f"KS test for Key {key} between {i} and {j}: Statistic={stat}, p-value={p_value}")
#             stat, p_value = stats.mannwhitneyu(group1, group2)


#             # Print the results
#             print(f"Mann test for Key {key} between {i} and {j}: Statistic={stat}, p-value={p_value}")

The Kolmogorov-Smirnov (KS) test can be used to check the normality of your data, and if you find that your data is not normally distributed, you might choose to use the Mann-Whitney U test, which is a non-parametric test for comparing two independent samples. Here's how you can use these tests:

In [ ]:
times_dict = {'AOI hit [slothStimulus-final (1) - sloth1]' : 0,
           'AOI hit [slothStimulus-final (1) - sloth2]': 2.5,
           'AOI hit [slothStimulus-final (1) - sloth3]': 5,
           'AOI hit [slothStimulus-final (1) - sloth4]': 7.5,
           'AOI hit [slothStimulus-final (1) - sloth5]': 10,
           'AOI hit [slothStimulus-final (1) - sloth6]': 12.5,
           'AOI hit [slothStimulus-final (1) - sloth7]': 15,
           'AOI hit [slothStimulus-final (1) - sloth8]': 17.5,
           'AOI hit [slothStimulus-final (1) - sloth9]': 20,
           'AOI hit [slothStimulus-final (1) - sloth10]': 22.5,
           'AOI hit [slothStimulus-final (1) - sloth11]': 25,
           'AOI hit [catStimulus-final (1) - cat1]': 0,
           'AOI hit [catStimulus-final (1) - cat2]': 2.5,
           'AOI hit [catStimulus-final (1) - cat3]': 5,
           'AOI hit [catStimulus-final (1) - cat4]': 7.5,
           'AOI hit [catStimulus-final (1) - cat5]': 10,
           'AOI hit [catStimulus-final (1) - cat6]': 12.5,
           'AOI hit [catStimulus-final (1) - cat7]': 15,
           'AOI hit [catStimulus-final (1) - cat8]': 17.5,
           'AOI hit [catStimulus-final (1) - cat9]': 20,
           'AOI hit [catStimulus-final (1) - cat10]': 22.5,
           'AOI hit [catStimulus-final (1) - cat11]': 25}

#     file_path1 = 'plusLenses/amblypiaStudySanaLeft4.xlsx'
dataExport = pd.read_excel('plusLenses/amblypiaStudySanaRight4.xlsx')
dfSaccade, dataExportWithoutCalibration = preprocessing(dataExport)

primarySaccadeTime, primarySaccadeIndex, secondarySaccadesTime, secondarySaccadesTimeAndAOI = primaryAndSecondarySaccade(dfSaccade, times_dict)
amplitudeBothEyesDict, amplitudeRightEyeDict, amplitudeLeftEyeDict = amplitudePrimarySaccade(dfSaccade, primarySaccadeIndex, dataExportWithoutCalibration)
amplitudeBothEyesDict, amplitudeRightEyeDict, amplitudeLeftEyeDict = secondarySaccadeTimeAmplitude(dfSaccade,dataExportWithoutCalibration, secondarySaccadesTime)
amplitudeBothEyesDict, amplitudeRightEyeDict, amplitudeLeftEyeDict = secondarySaccadeTimeandAOIAmplitude(dfSaccade,dataExportWithoutCalibration, secondarySaccadesTimeAndAOI)
saccade_latency = saccadeLatency(primarySaccadeTime, times_dict)
accuracy_dist = accuracyPrimarySaccade(primarySaccadeIndex, dfSaccade)
accuracy_dist = accuracySecondarySaccadeTime(secondarySaccadesTime, dfSaccade)
accuracy_dist = accuracySecondarySaccadeTimeandAOI(secondarySaccadesTimeAndAOI, dfSaccade)
correctiveSaccade = correctiveSaccadeTimeandAOI(secondarySaccadesTimeAndAOI, primarySaccadeIndex, dfSaccade)
correctiveSaccade = correctiveSaccadeTime(secondarySaccadesTime, primarySaccadeIndex, dfSaccade)



### Extracting relevant features from data metric

### TODO: Identify number of saccades in each AOI manually and where they are

In [ ]:
saccade_metrics_location = defaultdict(int)
saccade_metrics_number = defaultdict(int)

columns_to_keep = ['Gaze point X (MCSnorm)', 'Gaze point Y (MCSnorm)', 'Gaze point left X (MCSnorm)', 
                          'Gaze point left Y (MCSnorm)', 'Gaze point right X (MCSnorm)', 'Gaze point right Y (MCSnorm)']
for i in times_dict.keys():
    all_possible_hits = dfSaccade[dfSaccade[i] == np.max(dfSaccade[i])]

    total_saccades = 0
    df_saccades = pd.DataFrame()
    for index, row in all_possible_hits.iterrows(): 
        time = row['Eyetracker timestamp']
        #adding latency
        if (time/1000000 > (times_dict[i])+0.2) and (time/1000000 < (times_dict[i] + 2.5)):
            df_saccades = df_saccades.append(row[columns_to_keep], ignore_index=True)
        
            total_saccades +=1
    saccade_metrics_number[i] = total_saccades
    saccade_metrics_location[i] = df_saccades
  

saccade_metrics_location
saccade_metrics_number


In [ ]:
# dataMetricSaccades = dataMetric.iloc[:,list(range(520, dataMetric.shape[1]))]


In [ ]:
# dataMetricSaccades[['Number_of_saccades_in_AOI.bottomLeftCat1',
#        'Number_of_saccades_in_AOI.bottomLeftSloth',
#        'Number_of_saccades_in_AOI.bottomRightCat',
#        'Number_of_saccades_in_AOI.bottomRightCat2',
#        'Number_of_saccades_in_AOI.bottomRightSloth',
#        'Number_of_saccades_in_AOI.centerCat',
#        'Number_of_saccades_in_AOI.centerCat2',
#        'Number_of_saccades_in_AOI.centerSloth',
#        'Number_of_saccades_in_AOI.topCenterCat',
#        'Number_of_saccades_in_AOI.topCenterSloth',
#        'Number_of_saccades_in_AOI.topLeftCat',
#        'Number_of_saccades_in_AOI.topRightCat',
#        'Number_of_saccades_in_AOI.topRightSloth',
#        'Number_of_saccades_in_AOI.topRightSloth1',
#        'Time_to_entry_saccade.bottomLeftCat1']]

In [ ]:
# dataMetricSaccades['Time_to_entry_saccade.centerCat2']